In [1]:
# sync draft
import sys

sys.path.append("../")

import jijbench as jb
import jijzept as jz
import jijmodeling as jm

config_path = "/home/azureuser/.jij/config_prod.toml"

In [ ]:
import sys
import logging
logger =  logging.getLogger(__name__)
file_handler = logging.FileHandler("error.txt", mode='a', encoding='utf-8')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(logging.Formatter("%(message)s"))

logger.addHandler(file_handler)

In [54]:
def func_x():
    pass

def generate_warning_msg(metrics):
    return f"{func_x.__name__} cannot be calculated because \"{metrics}\" is not stored in table attribute of jijbench.Benchmark instance."

generate_warning_msg("obj")

'func_x cannot be calculated because "obj" is not stored in table attribute of jijbench.Benchmark instance.'

In [75]:
import numpy as np
import pandas as pd

df = pd.DataFrame([0, 1])

df.at[0] = object
df.at[:, 0] = np.array([0])
df

InvalidIndexError: slice(None, None, None)

In [8]:
if 0 < float("inf"):
    print("a")
else:
    

a


In [6]:
a = 1.

if a := {}:
    print("f")

In [1]:
import numpy as np

if np.nan:
    print("a")

np.log(0)

a


/tmp/ipykernel_10935/3952763878.py:6: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

In [52]:
if np.nan:
    print("a")

a


In [51]:
a = np.array([True,True])
print(a.any())
print(~a)
np.all(~a)

True
[False False]


False

In [25]:
np.isnan([[1, np.nan]]).all()

False

In [19]:
if np.isnan([1, np.nan]):
    pass

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

TypeError: 'int' object is not iterable

In [10]:
d = {"a": 1, "b": 2}
np.isnan(list(d.values()))

array([False, False])

In [62]:
d = jm.Placeholder("d", dim=2)
x = jm.Binary("x", shape=d.shape)
i = jm.Element("i", d.shape[0])
j = jm.Element("j", d.shape[1])

problem = jm.Problem("simple_problem")
problem += jm.Sum([i, j], d[i, j] * x[i, j])
problem += jm.Constraint("onehot1", x[i, :] == 1, forall=i)
problem += jm.Constraint("onehot2", x[:, j] == 1, forall=j)

instance_data = {"d": [[1, 10], [20, 2]]}


In [63]:
problem

In [64]:
sampler = jz.JijSASampler(config=config_path)
res = sampler.sample_model(problem, instance_data, search=True, num_search=5)
res

<APIStatus.SUCCESS: 'SUCCESS'>
DimodResponse(rec.array([([0, 0, 0, 0],   0.  , 1), ([1, 1, 1, 1], -38.35, 1),
           ([1, 0, 0, 1],  -1.5 , 1), ([0, 0, 0, 0],   0.  , 1),
           ([1, 0, 0, 1],  -0.3 , 1)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['x[0][0]', 'x[0][1]', 'x[1][0]', 'x[1][1]']), {'sampling_time': 1529.6208439394832, 'best_parameters': {'parameters': {'linear': {'onehot1': [[[0], -0.07500000000000001], [[1], -0.07500000000000001]], 'onehot2': [[[0], -0.07500000000000001], [[1], -0.07500000000000001]]}, 'quad': {'onehot1': [[[0], 0.07500000000000001], [[1], 0.07500000000000001]], 'onehot2': [[[0], 0.07500000000000001], [[1], 0.07500000000000001]]}}, 'constraint_violations': {'onehot1': 0.0, 'onehot2': 0.0}, 'objective': 3.0000000000000004}, 'measuring_time': {'solve': {'num_search': {}, 'compile': [], 'preprocess': [], 'generate_qubo': [], 'post_qubo': [], 'sample': [], 'fetch_solution': [], 'evaluate_soluti

In [65]:
decoded_samples = problem.decode(res, instance_data)
decoded_samples

DecodedSamples(feasibles = 2[/5], objective = 0.0, energy = -38.35)

In [66]:
decoded_samples.solutions

array([{'x': array([[0., 0.],
              [0., 0.]])}    , {'x': array([[1., 1.],
                                      [1., 1.]])}    ,
       {'x': array([[1., 0.],
              [0., 1.]])}    , {'x': array([[0., 0.],
                                      [0., 0.]])}    ,
       {'x': array([[1., 0.],
              [0., 1.]])}    ], dtype=object)

In [68]:
bench = jb.Benchmark(
    params={"search": [True], "num_search": [1, 3, 5]},
    solver="JijSASampler",
    problem=problem,
    instance_data=instance_data,
    jijzept_config=config_path
)
bench.run()

/home/azureuser/JijBenchmark/notebooks/../jijbench/experiment/experiment.py:83: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self._table.data.loc[self._table.current_index] = np.nan
/home/azureuser/JijBenchmark/notebooks/../jijbench/experiment/experiment.py:83: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  self._table.data.loc[self._table.current_index] = np.nan


In [83]:
feas = decoded_samples.feasibles()
feas.num_occurrences

array([1, 1])

In [69]:
bench.table[["objective", "num_occurances", "num_feasible", "num_samples", "onehot1_violations", "onehot2_violations"]]

,objective,num_occurances,num_feasible,num_samples,onehot1_violations,onehot2_violations
0,[0.0],[1],0,1,[2.0],[2.0]
1,"[0.0, 33.0, 3.0]","[1, 1, 1]",1,3,"[2.0, 2.0, 0.0]","[2.0, 2.0, 0.0]"
2,"[0.0, 33.0, 3.0, 0.0, 3.0]","[1, 1, 1, 1, 1]",2,5,"[2.0, 2.0, 0.0, 2.0, 0.0]","[2.0, 2.0, 0.0, 2.0, 0.0]"


In [17]:
import numpy as np

objective = np.array([2, 5])
const_vio = np.array([0, 1])
num_occurances = np.array([3, 3])

num_feas = num_occurances[const_vio == 0]
num_samples = []

array([3])

In [12]:
problem

In [ ]:
for x = 1

In [10]:
d = jm.Placeholder("d", dim=1)
x = jm.Binary("x", shape=(d.shape[0].set_latex("n")))
i = jm.Element("i", d.shape[0])
problem = jm.Problem("simple_problem")
problem += jm.Sum(i, d[i] * x[i])
problem += jm.Constraint("onehot", jm.Sum(i, d[i] * x[i]) == 1)
instance_data = {"d": [1, 2, 3]}

sampler = jz.JijSASampler(config="/home/azureuser/.jij/config_prod.toml")
res = sampler.sample_model(problem, instance_data, search=True, num_search=3)
res

<APIStatus.SUCCESS: 'SUCCESS'>
DimodResponse(rec.array([([0, 0, 0],   0.        , 1), ([0, 1, 0], -39.33333333, 1),
           ([1, 0, 0],  -0.33333333, 1)],
          dtype=[('sample', 'i1', (3,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(['x[0]', 'x[1]', 'x[2]']), {'sampling_time': 861.405860632658, 'best_parameters': {'parameters': {'linear': {'onehot': [[[], 116.0]]}, 'quad': {'onehot': [[[], 708.0]]}}, 'constraint_violations': {'onehot': 0.0}, 'objective': 1.0}, 'measuring_time': {'solve': {'num_search': {}, 'compile': [], 'preprocess': [], 'generate_qubo': [], 'post_qubo': [], 'sample': [], 'fetch_solution': [], 'evaluate_solution': [], 'schedule_multipliers': []}, 'system': {'post_problem_and_instance_data': 0.7900285720825195, 'request_queue': 0.35302233695983887, 'fetch_problem_and_instance': None, 'fetch_result': 4.561548709869385, 'download_solution': None, 'deserialize_solution': 0.000385284423828125}, 'total': 5.716908931732178}}, 'BINARY')

In [11]:
decoded_samples = problem.decode(res, instance_data)
decoded_samples

DecodedSamples(feasibles = 1[/3], objective = 0.0, energy = -39.333333333333336)

In [18]:
decoded_samples.num_occurrences

array([1, 1, 1])

In [2]:
num_sweeps_list = [100, 1000, 10000]
problem_name = "Knapsack"
problem, instance_data = jb.get_problem(problem_name), jb.get_instance_data(problem_name)

bench = jb.Benchmark(
    params={"num_sweeps": num_sweeps_list, "multipliers": [{"onehot_time": 0.5, "onehot_location": 0.5}]},
    solver="SASampler",
    problem=problem,
    instance_data=instance_data,
    jijzept_config="/home/azureuser/.config/jijzept/config.toml"
)
bench.run()
bench.evaluate(pr=0.99)

/home/azureuser/JijBenchmark/notebooks/../jijbench/evaluation/evaluation.py:132: RuntimeWarning: divide by zero encountered in log
  return np.log(1 - pr) / np.log(1 - ps) * x.execution_time
/home/azureuser/JijBenchmark/notebooks/../jijbench/evaluation/evaluation.py:132: RuntimeWarning: divide by zero encountered in log
  return np.log(1 - pr) / np.log(1 - ps) * x.execution_time
/home/azureuser/JijBenchmark/notebooks/../jijbench/evaluation/evaluation.py:132: RuntimeWarning: divide by zero encountered in log
  return np.log(1 - pr) / np.log(1 - ps) * x.execution_time
/home/azureuser/JijBenchmark/notebooks/../jijbench/evaluation/evaluation.py:132: RuntimeWarning: divide by zero encountered in log
  return np.log(1 - pr) / np.log(1 - ps) * x.execution_time
/home/azureuser/JijBenchmark/notebooks/../jijbench/evaluation/evaluation.py:132: RuntimeWarning: divide by zero encountered in log
  return np.log(1 - pr) / np.log(1 - ps) * x.execution_time
/home/azureuser/JijBenchmark/notebooks/../jij

,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,NaN,1.0,NaN,NaN,0.0,0.0
1,NaN,1.0,NaN,NaN,0.0,0.0
2,NaN,1.0,NaN,NaN,0.0,0.0
3,NaN,1.0,NaN,NaN,0.0,0.0
4,NaN,1.0,NaN,NaN,0.0,0.0
5,NaN,1.0,NaN,NaN,0.0,0.0
6,NaN,1.0,NaN,NaN,0.0,0.0
7,NaN,1.0,NaN,NaN,0.0,0.0
8,NaN,1.0,NaN,NaN,0.0,0.0
9,NaN,1.0,NaN,NaN,0.0,0.0


In [3]:
bench.table

,benchmark_id,experiment_id,run_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,knapsack_constraint_violations,knapsack_constraint_violation_min,knapsack_constraint_violation_mean,knapsack_constraint_violation_std,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,67a522ba-1685-4b4c-a190-4c493b063a2b,83b8bfd9-1128-4d23-b40f-4b371be356af,2022-07-01 09:07:26.039041,[-253.0],-253.0,-253.0,0.0,[-273.0],-273.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
1,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,67a522ba-1685-4b4c-a190-4c493b063a2b,77f5e54d-3b4c-4923-b29d-10271cbdee9e,2022-07-01 09:07:26.083607,[-339.0],-339.0,-339.0,0.0,[-339.0],-339.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
2,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,67a522ba-1685-4b4c-a190-4c493b063a2b,81820204-925c-48d4-9fb6-b1f1e7b6e03a,2022-07-01 09:07:26.127885,[-322.0],-322.0,-322.0,0.0,[-322.0],-322.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
3,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,84a1cd48-0e7f-41aa-8eb1-3a8fb6629a56,f6f4f906-d400-41a6-bb69-ba6dad378d49,2022-07-01 09:07:26.166263,[-280.86243514344096],-280.862435,-280.862435,0.0,[-281.063264],-281.063264,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
4,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,84a1cd48-0e7f-41aa-8eb1-3a8fb6629a56,b0af93da-198c-455c-aa96-e3c0b98aa34a,2022-07-01 09:07:26.210133,[-454.7114502383629],-454.71145,-454.71145,0.0,[-454.90585],-454.90585,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
5,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,84a1cd48-0e7f-41aa-8eb1-3a8fb6629a56,8ee39ddf-0a7b-4fc0-b391-f252c7cb7fff,2022-07-01 09:07:26.254971,[-422.24932580126915],-422.249326,-422.249326,0.0,[-422.257148],-422.257148,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
6,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,b3fda8ac-2018-4c58-a62a-6378dfbc5140,fd1854ff-08ae-47ba-a728-a8da01e033b8,2022-07-01 09:07:26.296323,[-420.0],-420.0,-420.0,0.0,[-420.0],-420.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
7,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,b3fda8ac-2018-4c58-a62a-6378dfbc5140,bc2ae102-1e75-48f2-ad48-aaa2f21b3145,2022-07-01 09:07:26.344992,[-683.0],-683.0,-683.0,0.0,[-683.0],-683.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
8,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,b3fda8ac-2018-4c58-a62a-6378dfbc5140,d99c22c6-f245-4284-a2c8-0593ed2932db,2022-07-01 09:07:26.395845,[-552.0],-552.0,-552.0,0.0,[-552.0],-552.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0
9,3ea55bf4-7fa1-467c-8487-d06b6b185bfc,f36c8a0b-80a3-4556-a9d2-11c27a252def,82ad60b2-725c-45ea-b7da-51fd10fc0fcd,2022-07-01 09:07:26.439430,[-595.0],-595.0,-595.0,0.0,[-595.0],-595.0,...,[0.0],0.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0


In [ ]:
import os
import json
import numpy as np
import pandas as pd

class Experiment1:
    def __init__(self, run_id=0, experiment_id=0, autosave=True, autosave_dir="."):
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._id_names = ["run_id", "experiment_id"]
        self.table = pd.DataFrame(columns=self._id_names)
        
        self._static_params = None
                
        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            self.benchmark_number = len([d for d in os.listdir(self.autosave_dir) if "benchmark" in d])
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_number}/tables"
        else:
            self.benchmark_number = None
            self.table_dir = None
    
    def __enter__(self, *args, **kwargs):
        self.start(args, kwargs)
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.end()
        
    def __next__(self):
        self.run_id += 1
        return self.run_id

    # 前処理
    def start(self, *args, **kwargs):
        self._static_params = kwargs

    #　後処理
    def end(self):
        cols = ["run_id", "experiment_id"]
        self.table[cols] = self.table[cols].astype(int)
        
    def insert_into_table(self, obj):
        self.table.loc[self.run_id, self._id_names] = [self.run_id, self.experiment_id]
        for k, v in self._static_params.items():
            self.table.loc[self.run_id, k] = v
        self.table.loc[self.run_id, "results"] = json.dumps(obj)
        next(self)
                
    def load(self, load_file=None):
        if self.autosave:
            self.table = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self.table = pd.read_csv(load_file)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.table.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.table.to_csv(save_file)
        

class Experiment2(pd.DataFrame):
    # temporary properties
    _internal_names = pd.DataFrame._internal_names + ["autosave", "autosave_dir", "id_names", "run_id", "experiment_id"]
    _internal_names_set = set(_internal_names)

    # normal properties
    _metadata = ["added_property"]

    @property
    def _constructor(self):
        return Experiment2

    def __init__(
        self,
        data=None,
        index=None,
        columns=None,
        dtype=None,
        copy=None,
        size=30,
        run_id=0,
        experiment_id=0,
        autosave=True,
        autosave_dir=".",
    ):
        data = [[np.nan, np.nan]] * size
        data = None
        self.id_names = ["run_id", "experiment_id"]
        self.id_names = None
        self.run_id = run_id
        self.experiment_id = experiment_id
        super().__init__(data=data, index=None, columns=self.id_names, dtype=None, copy=None)
        # self.loc[self.run_id, self.id_names] = [self.run_id, self.experiment_id]
        self.autosave = autosave
        self.autosave_dir = autosave_dir
    
    def start(self):
        pass
    
    def end(self):
        pass
        
    def next(self):
        self.run_id += 1
        self.loc[self.run_id, self.id_names] = [self.run_id, self.experiment_id]
    
    def insert_dict(self, obj):
        for k, v in obj.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            self.loc[self.run_id, k] = v

    def load(self, filepath=None):
        if self.autosave:
            self = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self = pd.read_csv(filepath)
        return self

    def save(self, filepath=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.to_csv(filepath)


class Experiment3:
    def __init__(self, run_id=0, experiment_id=0, autosave=True, autosave_dir="."):
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._id_names = ["run_id", "experiment_id"]
        self._table = None
        self._table_dtypes = {"run_id": int, "experiment_id": int}
            
        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            self.benchmark_id = len([d for d in os.listdir(self.autosave_dir) if "benchmark" in d])
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_id}/tables"
        else:
            self.benchmark_id = None
            self.table_dir = None
            
    @property
    def table(self):
        return self._table
    
    def __enter__(self, *args, **kwargs):
        self._table = pd.DataFrame(columns=self._id_names)
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        for (k, true), act in zip(self._table_dtypes.items(), self.table.dtypes):
            if true != act:
                self.table[k] = self.table[k].astype(true)
        
    def __next__(self):
        self.run_id += 1
        return self.run_id        
        
    def insert_into_table(self, obj):
        self.table.loc[self.run_id, self._id_names] = [self.run_id, self.experiment_id]
        for k, v in obj.items():
            if isinstance(v, dict):
                 v = json.dumps(v)
            
            self.table.loc[self.run_id, k] = v
            self._table_dtypes[k] = type(v)
        next(self)
                
    def load(self, load_file=None):
        if self.autosave:
            self.table = pd.read_csv(f"{self.table_dir}/table.csv")
        else:
            self.table = pd.read_csv(load_file)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self.table.to_csv(f"{self.table_dir}/table.csv")
        else:
            self.table.to_csv(save_file)


In [7]:
x = tuple((1, {"a": 1}))

len(x)

2

In [120]:
import numpy as np

ps = 1 - 1e-16

time = 1
result = np.log(1 - 0.99) / np.log(1 - ps) * time
result

4.147968546718736e+16

In [115]:
np.log(ps)

-1.1102230246251565e-16

9999999999999998.0

In [118]:
np.log(1-0.99)/np.log(ps)

4.147968546718736e+16

In [103]:
ps = np.nan

ps += 1e-16
ps

nan

In [83]:
np.log(1e-16)

-36.841361487904734

In [84]:
np.log(1-0.99)

-4.60517018598809

In [9]:
for xi in [1, 2]:
        print(xi)

1
2


In [14]:
x = [1, 2]
3 not in x

True

In [11]:
x, y = None, None

In [2]:
# Responseのサンプル
response_sample = {
    "type": "Response",
    "version": {"sampleset_schema": "3.1.0"},
    "num_variables": 20,
    "num_rows": 5,
    "sample_data": {
        "type": "array",
        "data": [[123424], [747616], [256132], [358510], [598528]],
        "data_type": "uint32",
        "shape": [5, 1],
        "use_bytes": False,
    },
    "sample_type": "int8",
    "sample_packed": True,
    "vectors": {
        "energy": {
            "type": "array",
            "data": [-207.0, -243.0, -272.0, -264.0, -157.0],
            "data_type": "float64",
            "shape": [5],
            "use_bytes": False,
        },
        "num_occurrences": {
            "type": "array",
            "data": [1, 1, 1, 1, 1],
            "data_type": "int64",
            "shape": [5],
            "use_bytes": False,
        },
    },
    "variable_labels": [
        "_leint_knapsack_constraint[0]",
        "_leint_knapsack_constraint[1]",
        "_leint_knapsack_constraint[2]",
        "_leint_knapsack_constraint[3]",
        "_leint_knapsack_constraint[4]",
        "_leint_knapsack_constraint[5]",
        "_leint_knapsack_constraint[6]",
        "_leint_knapsack_constraint[7]",
        "_leint_knapsack_constraint[8]",
        "_leint_knapsack_constraint[9]",
        "x[0]",
        "x[1]",
        "x[2]",
        "x[3]",
        "x[4]",
        "x[5]",
        "x[6]",
        "x[7]",
        "x[8]",
        "x[9]",
    ],
    "variable_type": "BINARY",
    "info": {
        "system": [],
        "sampling_time": 4100.264981389046,
        "execution_time": 751.1120289564133,
        "list_exec_times": {
            "type": "array",
            "data": [
                858.2137525081635,
                771.5118117630482,
                698.3112543821335,
                719.3121127784252,
                708.211213350296,
            ],
            "data_type": "float64",
            "shape": [5],
            "use_bytes": False,
        },
        "schedule": {
            "beta_max": 4.605170185988092,
            "beta_min": 4.652244285330389e-06,
            "num_sweeps": 1000,
        },
    },
}

In [166]:
# Experiment1を使ったサンプルコード
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

experiment = Experiment1()
for num_sweep in static_params["num_sweeps"]:
    for num_reads in static_params["num_reads"]:
        args = {"num_sweeps": num_sweep, "num_reads": num_reads}
        experiment.start(**args)
        for step in steps:
            # ここではsolve関数は上のresponse_sampleを返す想定
            # response_sample = solve()
            results = {"step": step,"response": response_sample}
            experiment.insert_into_table(results)
            
        experiment.end()


In [167]:
experiment.table

,run_id,experiment_id,num_sweeps,num_reads,results
0,0,0,10.0,1.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."
1,1,0,10.0,1.0,"{""step"": 1, ""response"": {""type"": ""Response"", ""..."
2,2,0,10.0,1.0,"{""step"": 2, ""response"": {""type"": ""Response"", ""..."
3,3,0,10.0,2.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."
4,4,0,10.0,2.0,"{""step"": 1, ""response"": {""type"": ""Response"", ""..."
5,5,0,10.0,2.0,"{""step"": 2, ""response"": {""type"": ""Response"", ""..."
6,6,0,10.0,3.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."
7,7,0,10.0,3.0,"{""step"": 1, ""response"": {""type"": ""Response"", ""..."
8,8,0,10.0,3.0,"{""step"": 2, ""response"": {""type"": ""Response"", ""..."
9,9,0,20.0,1.0,"{""step"": 0, ""response"": {""type"": ""Response"", ""..."


In [128]:
experiment.save()

In [105]:
# Experiment2を使ったサンプルコード
experiment = Experiment2()
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

experiment = Experiment2()
run_id = 0
experiment_id = 0
for num_sweep in static_params["num_sweeps"]:
    for num_reads in static_params["num_reads"]:
        args = {"num_sweeps": num_sweep, "num_reads": num_reads}
        experiment.start()
        for step in steps:
            # ここではsolve関数は上のresponse_sampleを返す想定
            # response_sample = solve()
            args["step"] = step
            args["response"] = response_sample
            experiment.insert_dict(args)
            experiment.next()
        experiment.end()



In [106]:
experiment

,run_id,experiment_id,num_sweeps,num_reads,step,response
0,0.0,0.0,10.0,1.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
1,1.0,0.0,10.0,1.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
2,2.0,0.0,10.0,1.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
3,3.0,0.0,10.0,2.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
4,4.0,0.0,10.0,2.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
5,5.0,0.0,10.0,2.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
6,6.0,0.0,10.0,3.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
7,7.0,0.0,10.0,3.0,1.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
8,8.0,0.0,10.0,3.0,2.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."
9,9.0,0.0,20.0,1.0,0.0,"{""type"": ""Response"", ""version"": {""sampleset_sc..."


In [225]:
# Experiment3を使ったサンプルコード
static_params = {
    "num_sweeps": [10, 20, 30],
    "num_reads": [1, 2, 3]
}
steps = range(3)

with Experiment3() as experiment:
    for num_sweeps in static_params["num_sweeps"]:
        for num_reads in static_params["num_reads"]:
            for step in steps:
                # ここではsolve関数は上のresponse_sampleを返す想定
                # response_sample = solve()
                
                # experiment.tableに登録するrecordを辞書型で作成
                record = {
                    "step": step,
                    "num_sweeps": num_sweeps,
                    "num_reads": num_reads,
                    "results": response_sample
                }
                experiment.insert_into_table(record)

In [226]:
experiment

In [109]:
df = pd.DataFrame()
df.loc["a", 0] = 1
df.loc["b", 1] = 1
df.loc["c", [0, 1]] = [1, 1]

df.dropna()

,0,1
c,1.0,1.0


In [116]:
df = Experiment2()
df.loc["a", 0] = 1
df.loc["b", 1] = 1
df.loc["c", [0, 1]] = [1, 1]

ValueError: cannot set a frame with no defined index and a scalar

In [3]:
import os
import json
from typing import Union
import pandas as pd


class Experiment:
    def __init__(
        self,
        run_id: int = 0,
        experiment_id: Union[int, str] = 0,
        benchmark_id: Union[int, str] = None,
        autosave: bool = True,
        autosave_dir: str = ".",
    ):
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.autosave = autosave
        self.autosave_dir = autosave_dir

        self._id_names = ["run_id", "experiment_id"]
        self._table = pd.DataFrame(columns=self._id_names)
        self._table_dtypes = {"run_id": int, "experiment_id": type(self.experiment_id)}

        if autosave:
            os.makedirs(autosave_dir, exist_ok=True)
            if benchmark_id is None:
                benchmark_id = sum(
                    [
                        os.path.isdir(f"{self.autosave_dir}/{d}")
                        for d in os.listdir(self.autosave_dir)
                        if "benchmark" in d
                    ]
                )
            self.benchmark_id = benchmark_id
            self.table_dir = f"{self.autosave_dir}/benchmark_{self.benchmark_id}/tables"
        else:
            self.benchmark_id = None
            self.table_dir = None

    @property
    def table(self):
        return self._table

    def __enter__(self, *args, **kwargs):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        pass

    def __next__(self):
        self.run_id += 1
        return self.run_id

    def insert_into_table(self, record, replace=True):
        self._table.loc[self.run_id, self._id_names] = [self.run_id, self.experiment_id]
        for k, v in record.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            if isinstance(v, list):
                v = str(v)

            self._table.loc[self.run_id, k] = v
            self._table[k] = self._table[k].astype(type(v))
            self._table_dtypes[k] = type(v)
        if replace:
            next(self)

    def load(self, load_file=None):
        if self.autosave:
            self._table = pd.read_csv(
                f"{self.table_dir}/experiment_id_{self.experiment_id}.csv", index_col=0
            )
        else:
            self._table = pd.read_csv(load_file, index_col=0)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self._table.to_csv(
                f"{self.table_dir}/experiment_id_{self.experiment_id}.csv"
            )
        else:
            self._table.to_csv(save_file)

In [25]:
experiment_id="sample"
benchmark_id="test"
experiment = Experiment(experiment_id=experiment_id, benchmark_id=benchmark_id)
experiment.table

,run_id,experiment_id


In [26]:
# Example 1
# 最も単純な使い方
# ユーザ定義のsolverの帰り値（何でも良い）
sample_response = {"hoge": {"fuga": 1}}
for param in [10, 100, 1000]:
    for step in range(3):
        # solverは上のsample_responseを返す想定
        # sample_response = solver()
        # experiment.tableに登録するrecordを辞書型で作成
        record = {
            "step": step,
            "param": param,
            "results": sample_response,
        }
        experiment.insert_into_table(record)
experiment.save()

In [145]:
import jijzept as jz
import jijmodeling as jm

d = jm.Placeholder("d", dim=1)
n = d.shape[0]

x = jm.Binary("x", shape=(n, ))

problem = jm.Problem("sample")

i = jm.Element("i", (0, n))

problem += jm.Sum(i, d[i]*x[i])
problem += jm.Constraint("onehot", x[:] == 1)
problem += jm.Constraint("onehot2", x[:] == 1)

data = {"d": [1, 2, -1, -3, -1]}

sampler = jz.JijSASampler(config="/home/azureuser/.config/jijzept/config.toml")
response = sampler.sample_model(problem, data, {'onehot': 4},  num_reads=5)


uploading instance ...
submitting query ...
submitted to the queue.
Your solution_id is d62dffd0616249dab9d702494579612c.


In [146]:
decoded = problem.decode(response, data, {})

In [3]:
print(decoded.__class__.__name__)

DecodedSamples


In [3]:
import openjij as oj

a = oj.Response

sampler = oj.SASampler()
pyq_obj = problem.to_pyqubo(data, {})
bqm = pyq_obj.compile().to_bqm(feed_dict={"onehot": 4})
sampler = oj.SASampler(num_reads=5)
response = sampler.sample(bqm)


In [328]:
import dimod

isinstance(response, dimod.SampleSet)
# print(response.energies)
print(response.record.energy)
print(response.record.num_occurrences)

[-3. -3. -3. -3. -3.]
[-3. -3. -3. -3. -3.]
[1 1 1 1 1]


In [331]:
x = ["a", "b", "c"]
x.remove("a")

x

None


['b', 'c']

In [61]:
import numpy
import pandas as pd
from typing import List

x = [[np.zeros((2, 2))]]

df = pd.DataFrame(x, columns=["a"])
df.astype({"a": object})

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'a' not found in columns."

In [76]:
_COLUMN_DTYPE = {
    "energies": object,
    "energy_min": float,
    "energy_mean": float,
    "energy_std": float,
    "objectives": object,
    "obj_min": float,
    "obj_mean": float,
    "obj_std": float,
    "num_feasible": int,
    "num_samples": int,
    "{const_name}_violations": object,
    "{const_name}_violation_min": float,
    "{const_name}_violation_mean": float,
    "{const_name}_violation_std": float
}

df = pd.DataFrame(columns=_TABLE_DTYPE.keys())
df.rename({df.columns[-1]: df.columns[-1].format(const_name="onehot")}, axis=1, inplace=True)
df

,energies,energy_min,energy_mean,energy_std,objectives,obj_min,obj_mean,obj_std,num_feasible,num_samples,onehot_violations


In [75]:
_TABLE_DTYPE

{'energies': object,
 'energy_min': float,
 'energy_mean': float,
 'energy_std': float,
 'objectives': object,
 'obj_min': float,
 'obj_mean': float,
 'obj_std': float,
 'num_feasible': int,
 'num_samples': int,
 '{const_name}_violations': object}

list

In [188]:
import os
import json
import uuid
import dimod
import numpy as np
import pandas as pd
from typing import Union

_ID_COLUMN_DTYPES = {
    "run_id": object,
    "experiment_id": object,
    "benchmark_id": object,
}

_FIXED_COLUMN_DTYPES = {
    
    "energy": object,
    "energy_min": float,
    "energy_mean": float,
    "energy_std": float,
    "objective": object,
    "obj_min": float,
    "obj_mean": float,
    "obj_std": float,
    "num_feasible": int,
    "num_samples": int,
}

_VIOLATION_COLUMN_DTYPES = {
    "{const_name}_violations": object,
    "{const_name}_violation_min": float,
    "{const_name}_violation_mean": float,
    "{const_name}_violation_std": float,
}


class Experiment:
    def __init__(
        self,
        experiment_id: Union[int, str] = None,
        benchmark_id: Union[int, str] = None,
        autosave: bool = True,
        autosave_dir: str = ".",
    ):        
        self._table = _Table(experiment_id=experiment_id, benchmark_id=benchmark_id)
        self._artifact = {}
        
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._dirs = _Dir(autosave=autosave, autosave_dir=autosave_dir)
        
    @property
    def run_id(self):
        return self._table.run_id
    
    @property
    def experiment_id(self):
        return self._table.experiment_id
    
    @property
    def benchmark_id(self):
        return self._table.benchmark_id
    
    @property
    def table(self):
        return self._table.data
    
    @property
    def artifact(self):
        return self._artifact

    def __enter__(self):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        pass
    
    def __next__(self):
        self._table.current_index += 1
        self._table.run_id = uuid.uuid4()
        return self.run_id

    def start(self):
        self._table.current_index = 0
        self._table.run_id = uuid.uuid4()
        self._table.experiment_id = uuid.uuid4() if self.experiment_id is None else self.experiment_id
        self._table.benchmark_id = uuid.uuid4() if self.benchmark_id is None else self.benchmark_id
        self._dirs.make_dirs(benchmark_id=self.benchmark_id)
        return self

    def end(self):
        pass

    def insert_into_table(self, record, replace=True):
        def _reconstruct_record():
            _new_record = {}
            for _k, _v in record.items():
                if isinstance(_v, dimod.SampleSet):                                        
                    _energies = _v.record.energy
                    _num_occurrences = response.record.num_occurrences
                    
                    _columns = experiment._table.get_energy_columns() + experiment._table.get_num_columns()
                    _values = [
                        list(_energies),
                        _energies.min(),
                        _energies.mean(),
                        _energies.std(),
                        list(_num_occurrences),
                        np.nan,
                        np.nan
                        ]
                    for _new_k, _new_v in zip(_columns, _values):
                        _new_record[_new_k] = _new_v      
                elif _v.__class__.__name__ == "DecodedSamples":
                    _energies = _v.energies
                    _objectives = _v.objectives
                    
                    _constraint_violations = {}
                    for _violation in _v.constraint_violations:
                        for _const_name, _value in _violation.items():
                            if _const_name in _constraint_violations.keys():
                                _constraint_violations[_const_name].append(_value)
                            else:
                                _constraint_violations[_const_name] = [_value]
                    
                    _columns = experiment._table.get_energy_columns()
                    _columns += experiment._table.get_objective_columns()
                    _columns += experiment._table.get_num_columns()
                    
                    _values = [
                        list(_energies),
                        _energies.min(),
                        _energies.mean(),
                        _energies.std(),
                        list(_objectives),
                        _objectives.min(),
                        _objectives.mean(),
                        _objectives.std(),
                        np.nan,
                        len(_v.feasibles()),
                        len(_v.data)
                    ]
                    for _const_name, _violation_values in _constraint_violations.items():
                        _violation_values = np.array(_violation_values)
                        _columns += self._table.rename_violation_columns(_const_name)
                        _values += [
                            list(_violation_values),
                            _violation_values.min(),
                            _violation_values.mean(),
                            _violation_values.std()
                            ]

                    for _new_k, _new_v in zip(_columns, _values):
                        _new_record[_new_k] = _new_v
                else:
                    _new_record[_k] = _v
            return _new_record
                
        index = self._table.current_index
        ids = self._table.get_id_columns()
        self._table.data.loc[index, ids] = [self.run_id, self.experiment_id, self.benchmark_id]
        
        record = _reconstruct_record()
        for k, v in record.items():
            if isinstance(v, dict):
                v = json.dumps(v)
            elif isinstance(v, list):
                v = str(v)

            self._table.data.loc[index, k] = v
            self._table.data[k] = self._table.data[k].astype(type(v))
    
        if replace:
            next(self)
            
    def update_artifact(self, results):
        self._artifact.update(results)

    def load(self, load_file=None):
        if self._autosave:
            self._table = pd.read_csv(
                f"{self.table_dir}/experiment_id_{self.experiment_id}.csv", index_col=0
            )
        else:
            self._table = pd.read_csv(load_file, index_col=0)

    def save(self, save_file=None):
        if self.autosave:
            os.makedirs(self.table_dir, exist_ok=True)
            self._table.to_csv(
                f"{self._table_dir}/experiment_id_{self.experiment_id}.csv"
            )
        else:
            self._table.to_csv(save_file)


class _Table:
    id_dtypes = {
        "run_id": object,
        "experiment_id": object,
        "benchmark_id": object,
    }
    energy_dtypes = {
        "energy": object,
        "energy_min": float,
        "energy_mean": float,
        "energy_std": float,
    }

    objective_dtypes = {
        "objective": object,
        "obj_min": float,
        "obj_mean": float,
        "obj_std": float,
    }

    num_dtypes = {
        "num_occurances": object,
        "num_feasible": int,
        "num_samples": int,
    }

    violation_dtypes = {
        "{const_name}_violations": object,
        "{const_name}_violation_min": float,
        "{const_name}_violation_mean": float,
        "{const_name}_violation_std": float,
    }
    
    __dtypes_names = [
        "id_dtypes",
        "energy_dtypes",
        "objective_dtypes",
        "num_dtypes",
        "violation_dtypes"
        ]
    
    def __init__(self, run_id=None, experiment_id=None, benchmark_id=None):
        columns = self.get_columns()
        self._data = pd.DataFrame(columns=columns)
        self._current_index = 0
        
        self.run_id = run_id
        self.experiment_id = experiment_id
        self.benchmark_id =  benchmark_id

    @property
    def data(self):
        return self._data
    
    @property
    def current_index(self):
        return self._current_index
    
    @current_index.setter
    def current_index(self, index):
        self._current_index = index
    
    def get_columns(self):
        c = []
        for name in self.__dtypes_names:
            c += list(getattr(self, name).keys())
        return c
    
    def get_id_columns(self):
        return list(self.id_dtypes.keys())
    
    def get_energy_columns(self):
        return list(self.energy_dtypes.keys())
    
    def get_objective_columns(self):
        return list(self.objective_dtypes.keys())
    
    def get_num_columns(self):
        return list(self.num_dtypes.keys())
    
    def get_violation_columns(self):
        return list(self.violation_dtypes.keys())
    
    def rename_violation_columns(self, const_name):
        columns = self.get_violation_columns()
        for i, c in enumerate(columns):
            columns[i] = c.format(const_name=const_name)
        return columns
    
    def get_dtypes(self):
        t = {}
        for name in self.__dtypes_names:
            t |= getattr(self, name)
        return t
    

class _Dir:
    def __init__(self, autosave, autosave_dir):
        self.autosave = autosave
        self.autosave_dir = autosave_dir
        
        self._table_dir = None
        self._artifact_dir = None
        
    @property
    def table_dir(self):
        return self._table_dir
    
    @property
    def artifact_dir(self):
        return self._artifact_dir
        
    def make_dirs(self, benchmark_id):
        if self.autosave:
            os.makedirs(self.autosave_dir, exist_ok=True)
            self._table_dir = f"{self.autosave_dir}/benchmark_{benchmark_id}/tables"
            self._artifact_dir = f"{self.autosave_dir}/benchmark_{benchmark_id}/artifacts"
    
            
def _insert_dimod_sampleset(experiment, response):
    index = 0
    energies = response.record.energy
    energy_min = energies.min()
    energy_mean = energies.mean()
    energy_std = energies.std()
    num_occurrences = response.record.num_occurrences
    
    columns = experiment._table.get_energy_columns() + experiment._table.get_num_columns()
    values = [
        energies,
        energy_min,
        energy_mean,
        energy_std,
        num_occurrences,
        np.nan,
        np.nan
        ]
    for c, v in zip(columns, values):
        print(index, c, v)
        experiment._table.data.loc[index, c] = json.dumps(list(v))
    
    return experiment
    

In [61]:
energies = response.record.energy

experiment = Experiment(benchmark_id="test")
experiment.start()
experiment._table.data.loc[0, "run_id"] = uuid.uuid4()
experiment._table.data.loc[0, "experiment_id"] = uuid.uuid4()
experiment._table.data.loc[0, "benchmark_id"] = "test"
experiment = _insert_dimod_sampleset(experiment, response)
experiment.table

0 energy [-3. -3. -3. -3. -3.]
0 energy_min -3.0
0 energy_mean -3.0
0 energy_std 0.0
0 num_occurances [1 1 1 1 1]
0 num_feasible nan
0 num_samples nan


,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,{const_name}_violations,{const_name}_violation_min,{const_name}_violation_mean,{const_name}_violation_std
0,a180c531-805d-4ce9-ad7f-2c7cda78a994,10bc44ff-3ab5-4c96-891d-7c123fc4a367,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
sample_response = {"hoge": {"fuga": 1}}

experiment = Experiment(benchmark_id="test")

with experiment.start() as e:
    for param in [10, 100, 1000]:
        for step in range(3):
            # solverは上のsample_responseを返す想定
            # sample_response = solver()
            # experiment.tableに登録するrecordを辞書型で作成
            record = {
                "step": step,
                "param": param,
                # "results": response,
                "results": decoded,
            }
            experiment.insert_into_table(record)

In [190]:
experiment.table

,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,...,step,param,onehot_violations,onehot_violation_min,onehot_violation_mean,onehot_violation_std,onehot2_violations,onehot2_violation_min,onehot2_violation_mean,onehot2_violation_std
0,05966a90-f271-4fd3-8300-f83eb8fa18b8,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,0,10,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
1,5b0bc8d0-4a84-4fcb-bdd1-fca260718699,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,1,10,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
2,3bde49d4-2c4f-4ad4-b9ef-5097b8814a01,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,2,10,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
3,86326e1a-d8a3-4621-b866-5e93b66f7d7e,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,0,100,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
4,16068876-a837-488c-a5eb-5923ecf5647b,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,1,100,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
5,766fbd23-6cb1-49f4-a86d-173973a47e7b,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,2,100,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
6,be87aafc-ff41-41ac-ad73-baca0f694e32,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,0,1000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
7,1d865ff2-dcf4-4cfe-9b2f-526cf0c57db7,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,1,1000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0
8,625fd3d2-b39d-4634-846e-1844a578389c,4608bda8-fb39-48d0-b6d4-e82493cbaa4e,test,"[-2.9999999999999982, -2.9999999999999982, -2....",-3.0,-3.0,0.0,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,...,2,1000,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.0,0.0


In [147]:
decoded.constraint_violations

array([{'onehot': 0.0, 'onehot2': 0.0}, {'onehot': 0.0, 'onehot2': 0.0},
       {'onehot': 0.0, 'onehot2': 0.0}, {'onehot': 0.0, 'onehot2': 0.0},
       {'onehot': 0.0, 'onehot2': 0.0}], dtype=object)

In [149]:
decoded.constraint_expr_value

array([{'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}},
       {'onehot': {(): 0.0}, 'onehot2': {(): 0.0}}], dtype=object)

In [116]:
for violations in decoded.constraint_violations:
    print(violations)

{'onehot': 0.0}


In [118]:
x = {"x": 1}
x

{'x': 1}

In [114]:
experiment.table

,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,{const_name}_violations,{const_name}_violation_min,{const_name}_violation_mean,{const_name}_violation_std,step,param
0,257f1517-d01a-4e4d-b908-3d99f4b4cdeb,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,0,10
1,2295f933-6535-49ad-8c45-0e485766cb7c,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,1,10
2,63f333c6-2093-4094-9ea8-bffaeb29143a,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,2,10
3,9b69d63a-c9f3-4a45-98ee-2c856681ef4b,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,0,100
4,a0d3dc90-9a76-4383-8818-d2183ab31bad,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,1,100
5,ba13a8d7-5e3e-4237-b00a-9c58ed482772,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,2,100
6,2a55f1a3-82d8-41ec-a4a4-a4e55e81f093,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,0,1000
7,ee2de150-1bfc-48da-913f-dfdcbf2fd1df,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,1,1000
8,2f3b01db-1a5a-4cb0-89f7-30bdc44edf0c,db525c95-3921-460a-ba54-b2744c18e733,test,"[-3.0, -3.0, -3.0, -3.0, -3.0]",-3.0,-3.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1]",NaN,NaN,NaN,NaN,NaN,NaN,2,1000


In [95]:
x = {"a":1 , "b": 2, "c": 3}

for k in ["a", "b"]:
    del x[k]
x

{'c': 3}

In [51]:
experiment.table

,run_id,experiment_id,benchmark_id,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,{const_name}_violations,{const_name}_violation_min,{const_name}_violation_mean,{const_name}_violation_std,step,param
0,ce6fe767-b049-4384-b954-010cfe4e6850,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,0,10
1,02f8c0a7-8c58-47f3-be1b-fba0e174238c,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,1,10
2,8f52ea2f-7b4e-4c3a-886f-22f14448070d,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,2,10
3,1d907818-c21a-4736-8f98-04b52c5bbe56,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,0,100
4,acc14cec-2f94-423a-9a77-0b379f59d2ca,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,1,100
5,7d21341a-f047-40f2-a6bd-7b6c4607f667,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,2,100
6,92e0206f-6bc1-4e0e-8c04-ffded51d2cdf,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,0,1000
7,c5153b07-6d66-45e6-b8e8-de1df8728109,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,1,1000
8,a8d3ad3d-a639-44ec-b6a6-75600bba0463,1d9c408d-4128-4fc4-9d0a-11d247706572,test,"[[[-3.0, -3.0, -3.0, -3.0, -3.0]]]",[-3.0],[-3.0],[0.0],NaN,NaN,NaN,NaN,"[[[1, 1, 1, 1, 1]]]",[nan],[nan],NaN,NaN,NaN,NaN,2,1000


In [239]:
import os
import json
import uuid
import pandas as pd
from typing import Dict, List, Union
from dataclasses import dataclass, field


class _TableColumn:
    id_dtypes = {
        "run_id": object,
        "experiment_id": object,
        "benchmark_id": object,
    }
    energy_dtypes = {
        "energy": object,
        "energy_min": float,
        "energy_mean": float,
        "energy_std": float,
    }

    objective_dtypes = {
        "objective": object,
        "obj_min": float,
        "obj_mean": float,
        "obj_std": float,
    }

    num_dtypes = {
        "num_feasible": int,
        "num_samples": int,
    }

    violation_dtypes = {
        "{const_name}_violations": object,
        "{const_name}_violation_min": float,
        "{const_name}_violation_mean": float,
        "{const_name}_violation_std": float,
    }
    
    __dtypes_names = [
        "id_dtypes",
        "energy_dtypes",
        "objective_dtypes",
        "num_dtypes",
        "violation_dtypes"
        ]
    
    def get_dtypes(self):
        t = {}
        for name in self.__dtypes_names:
            t |= getattr(self, name)
        return t
    
    
    def get_columns(self):
        c = []
        for name in self.__dtypes_names:
            c += list(getattr(self, name).keys())
        return c
    
    
    



['run_id', 'experiment_id', 'benchmark_id', 'energy', 'energy_min', 'energy_mean', 'energy_std', 'objective', 'obj_min', 'obj_mean', 'obj_std', 'num_feasible', 'num_samples', '{const_name}_violations', '{const_name}_violation_min', '{const_name}_violation_mean', '{const_name}_violation_std']


In [115]:
df = pd.DataFrame(columns=_get_columns()).astype(_get_column_dtypes())
df.dtypes

run_id                          object
experiment_id                   object
benchmark_id                    object
energy                          object
energy_min                     float64
energy_mean                    float64
energy_std                     float64
objective                       object
obj_min                        float64
obj_mean                       float64
obj_std                        float64
num_feasible                     int64
num_samples                      int64
{const_name}_violations         object
{const_name}_violation_min     float64
{const_name}_violation_mean    float64
{const_name}_violation_std     float64
dtype: object

In [109]:
_get_column_dtypes()

{'run_id': object,
 'experiment_id': object,
 'benchmark_id': object,
 'energy': object,
 'energy_min': float,
 'energy_mean': float,
 'energy_std': float,
 'objective': object,
 'obj_min': float,
 'obj_mean': float,
 'obj_std': float,
 'num_feasible': int,
 'num_samples': int,
 '{const_name}_violations': object,
 '{const_name}_violation_min': float,
 '{const_name}_violation_mean': float,
 '{const_name}_violation_std': float}

In [153]:
from typing import Tuple
from dataclasses import dataclass, asdict


@dataclass
class ColumnDtype:
    run_id: object = "a"
    experiment_id: type = (object,)
    benchmark_id: type = (object,)
    energy: type = (object,)
    energy_min: type = (float,)
    energy_mean: type = (float,)
    energy_std: type = (float,)
    objective: type = (object,)
    obj_min: type = (float,)
    obj_mean: type = (float,)
    obj_std: type = (float,)
    num_feasible: type = (int,)
    num_samples: type = (int,)
    
    def __post_init__(self, const_name):
        self.const = const_name
        
        setattr(self, "const", const_name)
        violations: Tuple[Tuple[str, type]] = (
            ("{const_name}_violations", object),
            ("{const_name}_violation_min", float),
            ("{const_name}_violation_mean", float),
            ("{const_name}_violation_std", float),
        )
        
    def get_columns(self):
        self.keys()
    

c = ColumnDtype("aaa")
c.const


AttributeError: 'ColumnDtype' object has no attribute 'const'

In [1]:
import sys

sys.path.append("../") 

import jijbench as jb

experiment = jb.Experiment()

In [8]:
experiment._table.get_dtypes()

{'run_id': object,
 'experiment_id': object,
 'benchmark_id': object,
 'timestamp': pandas._libs.tslibs.timestamps.Timestamp,
 'energy': object,
 'energy_min': float,
 'energy_mean': float,
 'energy_std': float,
 'objective': object,
 'obj_min': float,
 'obj_mean': float,
 'obj_std': float,
 'num_occurances': object,
 'num_feasible': int,
 'num_samples': int,
 '{const_name}_violations': object,
 '{const_name}_violation_min': float,
 '{const_name}_violation_mean': float,
 '{const_name}_violation_std': float}

In [213]:
json_str = "{\"a\": {\"b\": 1}}"
print(json_str)
print(json.loads(json_str))
print(type(json.loads(json_str)))

{'a': {"b": 1}}


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [221]:
import numpy as np
import pandas as pd

np.set_printoptions(threshold=np.inf)

df = pd.DataFrame([[1., 2., 3.], [2., 3., 4.]])
df.at[0, "list"] = object
df.at[0, "list"] = [[1., 2., 3.], [1., 2., 3.]]
df.at[0, "array"] = object
df.at[0, "array"] = np.random.normal(size=(10,5,4,3,2))
df.at[0, "dict"] = object
df.at[0, "dict"] = {"a": {"b": 1}}


df

,0,1,2,list,array,dict
0,1.0,2.0,3.0,"[[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]]","[[[0.6336372627527503, -0.8595895768799455], [...",{'a': {'b': 1}}
1,2.0,3.0,4.0,NaN,NaN,NaN


In [8]:
import openjij

openjij.SPIN

<Vartype.SPIN: frozenset({1, -1})>

In [2]:
class SolverAPIError(Exception):
    pass

class ParametervalidationError(Exception):
    pass

def fn():
    raise ParametervalidationError

x = 3
try:
    if x == 1:
        print(x)
    elif x == 2:
        raise RuntimeError("error")
    else:
        fn()
except RuntimeError as e:
    raise SolverAPIError(e)



ParametervalidationError: 

In [1]:
class SolverAPIError(Exception):
    pass

raise SolverAPIError("aaa")

SolverAPIError: aaa

In [220]:
class A:
    def __init__(self):
        self.a = 1
        
    def method(self, x):
        print(self)
        print(x)

A.method("a", 1)

a
1


In [217]:
df.loc[0].to_dict()

{0: 1.0,
 1: 2.0,
 2: 3.0,
 'list': [[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]],
 'array': array([[[[[ 8.58198797e-01, -5.00281048e-01,  7.18558388e-02,
            -8.94875125e-01],
           [ 9.63282311e-01,  5.51307617e-01,  2.23055923e+00,
             1.00004810e+00],
           [ 8.08116760e-01,  1.55381314e+00,  1.13366374e+00,
             1.03907525e+00]],
 
          [[-7.11809129e-02,  1.64354089e+00,  7.89208702e-01,
             4.76333197e-01],
           [-1.16166625e+00, -3.16467435e-01, -1.39335784e+00,
             7.42164051e-01],
           [ 9.03981399e-01,  1.50897340e+00, -8.70694518e-01,
            -7.99657673e-01]],
 
          [[ 5.91733770e-01, -7.09624713e-01, -4.82635512e-01,
            -7.23546706e-01],
           [-1.53004656e-01, -2.43822177e+00, -5.13552334e-01,
             7.06947621e-01],
           [-1.08872935e+00,  3.09911189e-01, -1.11693901e+00,
             1.21510595e+00]],
 
          [[ 8.92699706e-01, -1.07052825e+00, -2.79946423e-01,
          

In [203]:
print(type(df.loc[0, "list"]))
print(type(df.loc[0, "array"]))
print(type(df.loc[0, "dict"]))

<class 'list'>
<class 'numpy.ndarray'>
<class 'dict'>


In [222]:
df.to_csv("test.csv")

In [223]:
df = pd.read_csv("test.csv", index_col=0)


In [224]:
df

,0,1,2,list,array,dict
0,1.0,2.0,3.0,"[[1.0, 2.0, 3.0], [1.0, 2.0, 3.0]]",[[[ 0.63363726 -0.85958958]\n [ 1.75861345 -0...,{'a': {'b': 1}}
1,2.0,3.0,4.0,NaN,NaN,NaN


In [186]:
df.loc[0, "array"]

In [199]:
import re

def _replace_numpystr(x):
    try:
        return re.sub(
            r"\n+",
            "",
            re.sub(
                r" +",
                ", ",
                re.sub(
                    r"\[ +",
                    "[",
                    re.sub(
                        r" +\]", "]", re.sub(r"\.\]", ".0]", re.sub(r"\. ", ".0 ", x))
                    ),
                ),
            ),
        )
    except TypeError:
        return x

df = pd.read_csv("test.csv", index_col=0)
#df["array"] = df["array"].map(_replace)
#df

# regex = re.compile(r"\n+ ")
s = df.loc[0, "array"]
# re.findall(regex, s)

s = _replace_numpystr(s)
# regex = re.compile(r"\.[ \]]")
# s = re.sub(regex, ".")

series = df["array"].map(_replace_numpystr)
series = series.map(_loads)
series = series.map(np.array)


numpy.ndarray

In [189]:
np.array(json.loads(s)).shape

(10, 2, 5, 3, 4)

In [67]:
import json

def _loads(_x):
    try:
        return json.loads(_x)
    except (TypeError, json.JSONDecodeError):
        return _x
    
df = pd.read_csv("test.csv", index_col=0)
#print(df.loc[0, "list"])
#print(type(df.loc[0, "list"]))
#df["list"] = df["list"].map(_loads)
#print(type(df.loc[0, "list"]))

print()
print(df.loc[0, "array"])
s = df.loc[0, "array"].replace(". ", ".0 ").replace(".]", ".0]").replace("\n ", "*").replace(" ", ", ").replace("*", ", ")

print(s)
array = np.array(json.loads(s))





[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
[[[0.0, 0.0], , [0.0, 0.0]]
, [[0.0, 0.0], , [0.0, 0.0]]]


JSONDecodeError: Expecting value: line 1 column 15 (char 14)

In [201]:
json.loads("a")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [49]:
print(type(df.loc[0, "list"]))
print(type(df.loc[0, "array"]))
print(type(df.loc[0, "dict"]))

<class 'str'>
<class 'str'>
<class 'str'>


In [92]:
arraystr = df.loc[0, "list"]
arraystr = str([[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]])
array = np.fromstring(arraystr, sep=",")
array

/tmp/ipykernel_10963/683216600.py:3: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  array = np.fromstring(arraystr, sep=",")


array([], dtype=float64)

In [97]:
import re
s = df.loc[0, "list"]
# s = "[1, 2 ,3]"
s = arraystr
s = re.split("[\[\]]", s)
dim = 0
for c in s:
    if not c:
        dim += 1
    else:
        break
    
obj = []
for c in s:
    obj.append()



TypeError: list.append() takes exactly one argument (0 given)

In [100]:
s


['',
 '',
 '',
 '1, 2, 3',
 ', ',
 '1, 2, 3',
 '',
 ', ',
 '',
 '1, 2, 3',
 ', ',
 '1, 2, 3',
 '',
 '',
 '']

In [103]:
s = df.loc[0, "list"]
# s = "[1, 2 ,3]"
# s = arraystr
s = re.split("[\[\]]", s)
s

['', '', '1, 2, 3', ', ', '1, 2, 3', '', '']

In [12]:
import json

s = str([[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2]]]])
print(type(json.loads(s)))

x = np.zeros((2, 2))
json.loads(str(x.tolist()))

<class 'list'>


[[0.0, 0.0], [0.0, 0.0]]

In [4]:
s

NameError: name 's' is not defined

In [140]:
[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2]]]]

[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2]]]]

In [18]:

try:
    json.loads("str")
except (TypeError, json.JSONDecodeError):
    pass



TypeError: the JSON object must be str, bytes or bytearray, not float

In [19]:
json.loads("str")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [106]:
[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]], [[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]]]

[[[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]],
 [[[1, 2, 3], [1, 2, 3]], [[1, 2, 3], [1, 2, 3]]]]

In [86]:
s

['', '', '1, 2, 3', ', ', '1, 2, 3', '', '']

In [23]:
print(type(df.loc[0, 3]))
print(type(df.loc[0, 4]))

<class 'list'>
<class 'dict'>


In [24]:
import numpy as np

a = np.zeros(2)

type(a)

numpy.ndarray

In [25]:
df.at[3, 1] = np.zeros()

,0,1,2,3,4
0,1,2,3,"[1, 2, 3]",{'a': 1}
1,2,3,4,NaN,NaN


In [102]:
import jijbench as jb
import pandas as pd


class Evaluator:
    def __init__(self, experiment: jb.Experiment):
        self.experiment = experiment
    
    @property
    def table(self):
        return self.experiment.table
    
    @property
    def artifact(self):
        return self.experiment.artifact
    
    @property
    def ps(self):
        return self.success_probability
    
    @property
    def tts(self):
        return self.time_to_solution
    
    @property
    def fr(self):
        return self.feasible_rate
    
    @property
    def re(self):
        return self.residual_energy
    
    def calc_typical_metircs(self, opt_value=None, pr=0.99, expand=True):
        metrics = pd.DataFrame()
        metrics["success_probability"] = evaluator.ps(opt_value=opt_value, expand=expand)
        metrics["feasible_rate"] = evaluator.fr(expand=expand)
        metrics["residual_energy"] = evaluator.re(opt_value=opt_value, expand=expand)
        metrics["TTS(optimal)"] = evaluator.tts(opt_value=opt_value, pr=pr,solution="optimal", expand=expand)
        metrics["TTS(feasible)"] = evaluator.tts(pr=pr,solution="feasible", expand=expand)
        metrics["TTS(derived)"] = evaluator.tts(pr=pr,solution="derived", expand=expand)
        return metrics
        
    
    def apply(self, func, column, expand=True, axis=0):
        metrics = self.table.apply(func, axis=axis)
        if expand:
            self.table[column] = metrics
        return metrics
    
    def success_probability(self, opt_value=None, column="success_probability", expand=True):
        scorer = Evaluator.make_scorer(_Metrics.success_probability, opt_value=self._to_available(opt_value))
        return self.apply(func=scorer, column=column, expand=expand, axis=1)
    
    def time_to_solution(self, opt_value=None, pr=0.99, solution="optimal", column="TTS",expand=True):
        scorer = Evaluator.make_scorer(_Metrics.time_to_solution, opt_value=self._to_available(opt_value), pr=pr, solution=solution)
        return self.apply(func=scorer, column=f"{column}({solution})", expand=expand, axis=1)

    def feasible_rate(self, column="feasible_rate", expand=True):
        scorer = Evaluator.make_scorer(_Metrics.feasible_rate)
        return self.apply(func=scorer, column=column, expand=expand, axis=1)

    def residual_energy(self, opt_value=None, column="residual_energy", expand=True):
        scorer = Evaluator.make_scorer(_Metrics.residual_energy, opt_value=self._to_available(opt_value))
        return self.apply(func=scorer, column=column, expand=expand, axis=1)
        
    @staticmethod
    def make_scorer(score_func, **kwargs):
        return _Scorer(score_func, kwargs)
    
    @staticmethod
    def _to_available(value):
        if value is None:
            value = np.nan
        return value
    
class _Scorer:
    def __init__(self, score_func, kwargs):
        self._score_func = score_func
        self._kwargs = kwargs
        
    def __call__(self, x):
        return self._score_func(x, **self._kwargs)

class _Metrics:
    def time_to_solution(x, opt_value=None, pr=0.99, solution="optimal"):
        if solution == "optiaml":
            ps = _Metrics.success_probability(x, opt_value) + 1e-16
        elif solution == "feasible":
            ps = _Metrics.feasible_rate(x) + 1e-16
        elif solution == "derived":
            ps = _Metrics.success_probability(x, x.energy_min) + 1e-16
        else:
            ps = np.nan
        return np.log(1 - pr) / np.log(1 - ps) * x.execution_time if ps < pr else x.execution_time

    def success_probability(x, opt_value):
        return (x.energy <= opt_value).sum() / len(x.energy)

    def feasible_rate(x):
        return x.num_feasible / x.num_samples

    def residual_energy(x, opt_value):
        return (x.energy_mean - opt_value) / np.abs(opt_value)

In [105]:
import openjij as oj

sampler = oj.SASampler(num_reads=10)
experiment = jb.Experiment(autosave=False)
for _ in range(3):
    with experiment.start():
        response = sampler.sample_qubo({(0, 1): 1})
        experiment.store({"result": response})
# experiment.table.dropna(axis="columns", inplace=True)

In [84]:
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,sampling_time,execution_time
0,b392168c-455a-480f-ab58-dce81f0d77af,24e98bf7-9e0f-4024-b700-f944dc5b16c2,fcfa2216-ee2c-41d1-ac23-01446fe68f00,2022-03-15 02:40:59.937822,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",NaN,NaN,2226.732671,195.232965
1,63b6edd6-a28c-4387-b82c-de1be46f6ac8,24e98bf7-9e0f-4024-b700-f944dc5b16c2,fcfa2216-ee2c-41d1-ac23-01446fe68f00,2022-03-15 02:40:59.946139,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",NaN,NaN,2440.435812,213.653222
2,4af83001-a2ff-490f-8c33-20cb0102a7a2,24e98bf7-9e0f-4024-b700-f944dc5b16c2,fcfa2216-ee2c-41d1-ac23-01446fe68f00,2022-03-15 02:40:59.956266,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",NaN,NaN,2984.943800,254.993979


In [85]:
evaluator = Evaluator(experiment)
evaluator.feasible_rate()

0   NaN
1   NaN
2   NaN
dtype: float64

In [120]:
import jijmodeling as jm

d = jm.Placeholder("d")
x = jm.Binary("x", shape=(2,))
problem = jm.Problem("sample")
problem += x[0] + d * x[1]
problem += jm.Constraint("onehot", x[:] == 1)
ph_value = {"d": 2}
pyq_obj = problem.to_pyqubo(ph_value=ph_value)
pyq_model = pyq_obj.compile()
sampler = oj.SASampler(num_reads=10)
experiment = jb.Experiment(autosave=False)
for _ in range(3):
    with experiment.start():
        bqm = pyq_model.to_bqm(feed_dict={"onehot": 1})
        response = sampler.sample(bqm)
        decoded = problem.decode(response, ph_value=ph_value)
        experiment.store({"result": decoded})
        # experiment.store({"response":response, "result": decoded})
# experiment.table.dropna(axis=1, inplace=True)

In [121]:
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,obj_std,num_occurances,num_feasible,num_samples,sampling_time,execution_time,onehot_violations,onehot_violation_min,onehot_violation_mean,onehot_violation_std
0,30aaa028-f982-4bf9-8711-140da71751bb,99b2c721-3dc2-41bc-8c5b-4dafabfe8beb,1235dc2b-0ad7-46a3-9a25-b629fc9d8934,2022-03-15 03:12:04.506527,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,...,0.458258,NaN,7,10,NaN,NaN,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.3,0.458258
1,53ab9cbb-7456-44c9-b937-35c6fb4ef8b8,99b2c721-3dc2-41bc-8c5b-4dafabfe8beb,1235dc2b-0ad7-46a3-9a25-b629fc9d8934,2022-03-15 03:12:04.523332,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,...,0.489898,NaN,4,10,NaN,NaN,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...",0.0,0.6,0.489898
2,4eeba1f8-a798-44db-adec-e1fd4e2901a0,99b2c721-3dc2-41bc-8c5b-4dafabfe8beb,1235dc2b-0ad7-46a3-9a25-b629fc9d8934,2022-03-15 03:12:04.536512,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,...,0.500000,NaN,5,10,NaN,NaN,"[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...",0.0,0.5,0.500000


In [23]:
experiment.table.energy.map(lambda x: (x <= 0).sum())

0    10
1    10
2    10
Name: energy, dtype: int64

In [133]:
evaluator = Evaluator(experiment)

In [ ]:
evaluator.tts(1, solution="optimal")
evaluator.tts(solution="feasible")
evaluator.tts(solution="derived")
evaluator.success_probability(1)
evaluator.feasible_rate()
evaluator.residual_energy(1)

In [124]:
metrics = evaluator.calc_typical_metircs(1)

In [112]:
evaluator.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,num_feasible,num_samples,sampling_time,execution_time,TTS(optimal),TTS(feasible),TTS(derived),success_probability,feasible_rate,residual_energy
0,b53bfb2a-51e2-48dd-9a3b-fc522b430590,782afffd-74a7-4d8b-9096-cda5b16cee28,7a474766-cefb-402f-84f6-5ce2cab5c592,2022-03-15 02:51:13.132324,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,2668.146975,236.834213,236.834213,236.834213,236.834213,1.0,NaN,-1.0
1,5f0f5acb-77fb-4623-b832-76f03e5ec3ad,782afffd-74a7-4d8b-9096-cda5b16cee28,7a474766-cefb-402f-84f6-5ce2cab5c592,2022-03-15 02:51:13.156439,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,2632.146701,232.364237,232.364237,232.364237,232.364237,1.0,NaN,-1.0
2,d3655d01-10e8-4100-a75a-40048c4f8df8,782afffd-74a7-4d8b-9096-cda5b16cee28,7a474766-cefb-402f-84f6-5ce2cab5c592,2022-03-15 02:51:13.166168,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,2137.338743,192.063302,192.063302,192.063302,192.063302,1.0,NaN,-1.0


In [9]:
import numpy as np

pr = 0.99
ps = 0.999
execution_time = 150

np.log(1 - pr) / np.log(1 - ps) * execution_time if ps < pr else execution_time

100.0

In [2]:
import numpy as np
import pandas as pd


df = pd.DataFrame()
df.at[0, "a"] = object
df.at[0, "a"] = [1, 2, 3]
df.at[1, "a"] = object
df.at[1, "a"] = []
df.at[2, "a"] = object
df.at[2, "a"] = [3, 4]
df.at[3, "a"] = object
df.at[3, "a"] = [3]

df.at[0, "b"] = np.nan
df.at[1, "b"] = np.nan
df.at[2, "b"] = np.nan
df.at[3, "b"] = 1

df.to_csv("test.csv")

df

,a,b
0,"[1, 2, 3]",NaN
1,[],NaN
2,"[3, 4]",NaN
3,[3],1.0


In [3]:
df = pd.read_csv("test.csv", index_col=0)
df

,a,b
0,"[1, 2, 3]",NaN
1,[],NaN
2,"[3, 4]",NaN
3,[3],1.0


In [12]:
df = pd.read_csv("table2.csv")
df

Unnamed: 0  \
0 2a22467e-e3c2-415d-8b4e-c8d77d6185cf d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:16.660470   
  e359070f-52b4-4827-a9c0-ce5747f991eb d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:17.688829   
  1ac8a012-b6ae-4ef4-ae90-b883f54a644f d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:19.063011   
  fd74a333-e7db-4b6a-937e-47496661e368 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:20.082652   
  5a2837e8-d24d-4d3e-8cac-4f7196694553 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:21.381275   
  6097cdbe-269b-4589-aceb-723dee4ee1fa d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:22.248199   
  66920313-7c0b-4b76-9ba5-2add07180b2d d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:22.626656   
  739c9570-e443-488f-8cd1-3175598c5cc6 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:23.040365   
  b4f8759f-c247-4971-a595-43889296fd0b d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:23.424877   
  f50b8040-cffa-47e4-ac6d-fe7e0a060fe0 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:23.798238   
  4a7f43f3-14c5-41a0-aa58-6dd075b56276 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:24.171440   
  8e1db546-1c04-4dbe-8941-a7d4e393077d d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:24.552216   
  c418eaa2-a946-41d9-8fe1-4bd85ea76109 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:24.935836   
  6b267616-f07a-44ed-9333-ff6559bbf35a d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:25.305557   
  c694129f-a56f-4928-a2f4-ea015f526d18 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:25.684207   
  d6956f79-d1f3-40d1-823c-1d0fed6dc42e d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33  2022-03-15 04:08:26.059501   

                                                                                                                  run_id  \
0 2a22467e-e3c2-415d-8b4e-c8d77d6185cf d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.0   
  e359070f-52b4-4827-a9c0-ce5747f991eb d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.0   
  1ac8a012-b6ae-4ef4-ae90-b883f54a644f d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.0   
  fd74a333-e7db-4b6a-937e-47496661e368 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.0   
  5a2837e8-d24d-4d3e-8cac-4f7196694553 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.5   
  6097cdbe-269b-4589-aceb-723dee4ee1fa d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.5   
  66920313-7c0b-4b76-9ba5-2add07180b2d d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.5   
  739c9570-e443-488f-8cd1-3175598c5cc6 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     0.5   
  b4f8759f-c247-4971-a595-43889296fd0b d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     1.0   
  f50b8040-cffa-47e4-ac6d-fe7e0a060fe0 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     1.0   
  4a7f43f3-14c5-41a0-aa58-6dd075b56276 d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     1.0   
  8e1db546-1c04-4dbe-8941-a7d4e393077d d1c5d161-c2d8-43dc-86a9-70806bde9c65 3090b40f-1e72-454a-b1c5-3bb88310eb33     1.0   
  c418eaa2-a946-41d9-8fe1-4bd85ea76109 d1c5d161-c2d8-43dc-86a9-70806b

In [19]:
with open("table2.csv", "r") as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())
    print(f.readline())
    f.readline() + ",,,,"

<class 'str'>
0,2a22467e-e3c2-415d-8b4e-c8d77d6185cf,d1c5d161-c2d8-43dc-86a9-70806bde9c65,3090b40f-1e72-454a-b1c5-3bb88310eb33,2022-03-15 04:08:16.660470,0.0,0.0,100,[]

0,e359070f-52b4-4827-a9c0-ce5747f991eb,d1c5d161-c2d8-43dc-86a9-70806bde9c65,3090b40f-1e72-454a-b1c5-3bb88310eb33,2022-03-15 04:08:17.688829,0.0,0.5,100,[]

0,1ac8a012-b6ae-4ef4-ae90-b883f54a644f,d1c5d161-c2d8-43dc-86a9-70806bde9c65,3090b40f-1e72-454a-b1c5-3bb88310eb33,2022-03-15 04:08:19.063011,0.0,1.0,100,[]

0,fd74a333-e7db-4b6a-937e-47496661e368,d1c5d161-c2d8-43dc-86a9-70806bde9c65,3090b40f-1e72-454a-b1c5-3bb88310eb33,2022-03-15 04:08:20.082652,0.0,1.5,100,[]

0,5a2837e8-d24d-4d3e-8cac-4f7196694553,d1c5d161-c2d8-43dc-86a9-70806bde9c65,3090b40f-1e72-454a-b1c5-3bb88310eb33,2022-03-15 04:08:21.381275,0.5,0.0,100,[]

0,6097cdbe-269b-4589-aceb-723dee4ee1fa,d1c5d161-c2d8-43dc-86a9-70806bde9c65,3090b40f-1e72-454a-b1c5-3bb88310eb33,2022-03-15 04:08:22.248199,0.5,0.5,100,[0.6048938948983693],0.6048938948983693,0.604893894898

In [51]:
import numpy as np
import pandas as pd
import jijbench as jb
    
df = pd.DataFrame(np.ones((10, 2)), columns=["a", "b"])
df

,a,b
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,1.0


In [50]:
def score1(x):
    return x.a + x.b

df["score1"] = df.apply(score1, axis=1)
df

,a,b,score1
0,1.0,1.0,2.0
1,1.0,1.0,2.0
2,1.0,1.0,2.0
3,1.0,1.0,2.0
4,1.0,1.0,2.0
5,1.0,1.0,2.0
6,1.0,1.0,2.0
7,1.0,1.0,2.0
8,1.0,1.0,2.0
9,1.0,1.0,2.0


In [33]:
def score2(x, c1=1, c2=1):
    return x.a + x.b + c1 + c2

df["score2"] = df.apply(score2, axis=1)
df

,a,b,score1,score2
0,1.0,1.0,2.0,4.0
1,1.0,1.0,2.0,4.0
2,1.0,1.0,2.0,4.0
3,1.0,1.0,2.0,4.0
4,1.0,1.0,2.0,4.0
5,1.0,1.0,2.0,4.0
6,1.0,1.0,2.0,4.0
7,1.0,1.0,2.0,4.0
8,1.0,1.0,2.0,4.0
9,1.0,1.0,2.0,4.0


In [34]:
func = jb.Evaluator.make_scorer(score2, c1=3, c2=5)
df["score2"] = df.apply(func, axis=1)
df

,a,b,score1,score2
0,1.0,1.0,2.0,10.0
1,1.0,1.0,2.0,10.0
2,1.0,1.0,2.0,10.0
3,1.0,1.0,2.0,10.0
4,1.0,1.0,2.0,10.0
5,1.0,1.0,2.0,10.0
6,1.0,1.0,2.0,10.0
7,1.0,1.0,2.0,10.0
8,1.0,1.0,2.0,10.0
9,1.0,1.0,2.0,10.0


In [39]:
experiment = jb.Experiment()

experiment.table["score1"] = np.ones(10)
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_feasible,num_samples,sampling_time,execution_time,score1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [1]:
import jijbench as jb
import openjij as oj
import jijmodeling as jm

experiment = jb.Experiment()

sampler = oj.SASampler(num_reads=10)
for _ in range(3):
    with experiment.start():
        response = sampler.sample_qubo({(0, 1): 1})
        experiment.store({"result": response})

experiment.table


,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_reads,num_sweeps,num_feasible,num_samples,sampling_time,execution_time
0,f0d4edf2-2d94-4733-8e53-1f72539d9d80,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.180510,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1701.925881,153.032225
1,d0ee043f-8906-4d0d-ab10-4cb580d67ddd,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.196524,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1667.825505,150.832441
2,2303471a-abe2-4c2e-87f9-2508177c5ac8,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.208312,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1673.426479,151.182339


In [16]:
import pandas as pd

df = pd.DataFrame()
df.loc[:, "a"] = [0] * 10
df.a.values[-1]

0

In [3]:
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_reads,num_sweeps,num_feasible,num_samples,sampling_time,execution_time
0,f0d4edf2-2d94-4733-8e53-1f72539d9d80,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.180510,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1701.925881,153.032225
1,d0ee043f-8906-4d0d-ab10-4cb580d67ddd,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.196524,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1667.825505,150.832441
2,2303471a-abe2-4c2e-87f9-2508177c5ac8,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.208312,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1673.426479,151.182339


In [4]:
evaluator = jb.Evaluator(experiment)

In [9]:
def func(x, pr=0.99, a=1, b=1):
    return x.num_reads + a + b

evaluator.apply(func, column="new_score2", pr=0.5)

0    12
1    12
2    12
dtype: int64

In [11]:
evaluator.calc_typical_metrics()

/home/azureuser/JijBenchmark/jijbench/evaluation/evaluation.py:132: RuntimeWarning: divide by zero encountered in log
  return np.log(1 - pr) / np.log(1 - ps) * x.execution_time


,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,NaN,NaN,NaN,NaN,NaN,0.0
1,NaN,NaN,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,NaN,NaN,0.0


In [10]:
evaluator.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_reads,num_sweeps,num_feasible,num_samples,sampling_time,execution_time,new_score
0,f0d4edf2-2d94-4733-8e53-1f72539d9d80,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.180510,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1701.925881,153.032225,12
1,d0ee043f-8906-4d0d-ab10-4cb580d67ddd,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.196524,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1667.825505,150.832441,12
2,2303471a-abe2-4c2e-87f9-2508177c5ac8,5976a586-da87-4448-a8bb-951c084e8456,6f50eaa2-5987-47ad-9ded-758c9744cecd,2022-03-25 03:11:27.208312,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.0,NaN,NaN,NaN,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",10,1000,NaN,NaN,1673.426479,151.182339,12


In [13]:
"""
成功確率：
opt_valueは定義されてない時はNaNが返ってくる

TTS(optimal)：
成功確率が0の時はINF
成功確率がNaNは時はNaN
"""
evaluator.calc_typical_metrics(-1)

/home/azureuser/JijBenchmark/jijbench/evaluation/evaluation.py:115: RuntimeWarning: divide by zero encountered in double_scalars
  np.log(1 - pr) / np.log(1 - ps) * x.execution_time
/home/azureuser/JijBenchmark/jijbench/evaluation/evaluation.py:115: RuntimeWarning: divide by zero encountered in log
  np.log(1 - pr) / np.log(1 - ps) * x.execution_time


,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,0.0,0.4,2.0,-inf,1342.824013,0.0
1,0.0,0.1,2.0,-inf,6551.163392,0.0
2,0.0,0.5,2.0,-inf,995.597243,0.0


In [12]:
evaluator.time_to_solution(-1)

/home/azureuser/JijBenchmark/jijbench/evaluation/evaluation.py:115: RuntimeWarning: divide by zero encountered in double_scalars
  np.log(1 - pr) / np.log(1 - ps) * x.execution_time


0   -inf
1   -inf
2   -inf
dtype: float64

In [11]:
evaluator.success_probability(-1)

0    0.0
1    0.0
2    0.0
dtype: float64

In [7]:
import jijbench as jb
import openjij as oj
import jijmodeling as jm

d = jm.Placeholder("d")
x = jm.Binary("x", shape=(2,))
problem = jm.Problem("sample")
problem += x[0] + d * x[1]
problem += jm.Constraint("onehot", x[:] == 1)
ph_value = {"d": 2}
pyq_obj = problem.to_pyqubo(ph_value=ph_value)
pyq_model = pyq_obj.compile()
sampler = oj.SASampler(num_reads=10)

experiment = jb.Experiment(autosave=False)
for _ in range(3):
    with experiment.start():
        bqm = pyq_model.to_bqm(feed_dict={"onehot": 1})
        response = sampler.sample(bqm)
        decoded = problem.decode(response, ph_value=ph_value)
        experiment.store({"response": response, "result": decoded})
        
experiment.table.iloc[:, -10:]

,num_reads,num_sweeps,num_feasible,num_samples,sampling_time,execution_time,onehot_violations,onehot_violation_min,onehot_violation_mean,onehot_violation_std
0,10,1000,4,10,1676.825806,148.951914,"[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,0.6,0.489898
1,10,1000,1,10,1674.826257,149.882399,"[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,0.9,0.300000
2,10,1000,5,10,1662.224531,149.852317,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.5,0.500000


In [8]:
evaluator = jb.Evaluator(experiment)

In [10]:
evaluator.calc_typical_metrics()

/home/azureuser/JijBenchmark/jijbench/evaluation/evaluation.py:115: RuntimeWarning: divide by zero encountered in log
  np.log(1 - pr) / np.log(1 - ps) * x.execution_time


,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,NaN,0.4,NaN,NaN,1342.824013,0.0
1,NaN,0.1,NaN,NaN,6551.163392,0.0
2,NaN,0.5,NaN,NaN,995.597243,0.0


In [11]:
import numpy as np

np.log(1 - 0.99) / np.log(1 - 1.0)

/tmp/ipykernel_15702/3814384496.py:3: RuntimeWarning: divide by zero encountered in log
  np.log(1 - 0.99) / np.log(1 - 1.0) * 250


0.0

In [10]:
evaluator.table.energy[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [14]:
evaluator.table.columns

Index(['run_id', 'experiment_id', 'benchmark_id', 'timestamp', 'energy',
       'energy_min', 'energy_mean', 'energy_std', 'objective', 'obj_min',
       'obj_mean', 'obj_std', 'num_occurances', 'num_feasible', 'num_samples',
       'sampling_time', 'execution_time', 'onehot_violations',
       'onehot_violation_min', 'onehot_violation_mean', 'onehot_violation_std',
       'success_probability', 'feasible_rate', 'residual_energy',
       'TTS(optimal)', 'TTS(feasible)', 'TTS(derived)'],
      dtype='object')

In [14]:
from typing import Union, Tuple, Dict

a = ("str", {})
b = ("str", {}}

isinstance(a, tuple)

True

In [17]:
def func():
    return (1, 2)

func()

SyntaxError: can't use starred expression here (2492818244.py, line 2)

In [40]:
import jijbench as jb
import openjij as oj
import jijmodeling as jm
import numpy as np

d = jm.Placeholder("d")
x = jm.Binary("x", shape=(2,))
problem = jm.Problem("sample")
problem += x[0] + d * x[1]
problem += jm.Constraint("onehot", x[:] == 1)
ph_value = {"d": 2}
pyq_obj = problem.to_pyqubo(ph_value=ph_value)
pyq_model = pyq_obj.compile()
sampler = oj.SASampler(num_reads=10)

experiment = jb.Experiment(experiment_id="test", benchmark_id=1)
params = np.arange(0, 50, 5)
count = 0
for p_1 in params:
    for p_g in params:
        with experiment:
            bqm = pyq_model.to_bqm(feed_dict={"onehot": 1})
            response = sampler.sample(bqm)
            decoded = problem.decode(response, ph_value=ph_value)
            if count > 20:
                # fes_obj = [decoded.feasibles()[i][3] for i in range(len(decoded.feasibles()))]
                fes_obj = list(np.random.randn(10))
            elif count == 20:
                fes_obj = [np.random.randn()] 
            else:
                fes_obj = []
            feas_obj_min  = np.min(fes_obj) if count >= 20 else np.nan
            feas_obj_max  = np.max(fes_obj) if count >= 20 != 0 else np.nan
            feas_obj_std  = np.std(fes_obj) if count >= 20 != 0 else np.nan
            feas_obj_mean = np.average(fes_obj) if count >= 20 != 0 else np.nan
            experiment.store({"decoded": decoded})
            experiment.store({
                                "one-hot": p_1,
                                "group_num_people": p_g,
                                "feas_obj": fes_obj,
                                "feas_obj_min": feas_obj_min,
                                "feas_obj_max": feas_obj_max,
                                "feas_obj_std": feas_obj_std,
                                "feas_obj_mean": feas_obj_mean
                                })
        
        count += 1


In [41]:
problem.name

'sample'

In [2]:
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,onehot_violation_min,onehot_violation_mean,onehot_violation_std,one-hot,group_num_people,feas_obj,feas_obj_min,feas_obj_max,feas_obj_std,feas_obj_mean
0,4e81c477-71c3-480b-bdb3-973741326e86,test,1,2022-03-22 17:07:30.583433,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,...,0.0,0.5,0.500000,0,0,[],NaN,NaN,NaN,NaN
1,147d905e-f30e-42c5-ada2-23780429edac,test,1,2022-03-22 17:07:30.609791,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",0.0,...,0.0,0.6,0.489898,0,5,[],NaN,NaN,NaN,NaN
2,c965fe6c-dd8a-41c8-a259-3fda687b0917,test,1,2022-03-22 17:07:30.630044,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,...,0.0,0.3,0.458258,0,10,[],NaN,NaN,NaN,NaN
3,8f6e8db0-92c3-45e7-a68a-60bed8af9b78,test,1,2022-03-22 17:07:30.649832,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...",0.0,...,0.0,0.3,0.458258,0,15,[],NaN,NaN,NaN,NaN
4,716a25c6-23f3-4e6d-ba88-9c80170d0d26,test,1,2022-03-22 17:07:30.668963,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,...,0.0,0.5,0.500000,0,20,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,abf91fcb-9a36-4e90-aaba-b1aebda84136,test,1,2022-03-22 17:07:32.403204,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",0.0,...,0.0,0.3,0.458258,45,25,"[0.1794475112135184, -0.08708276341715952, -0....",-1.904287,1.494498,0.987479,0.085032
96,c8264a09-23c5-4e32-b3ff-75108bcd6d7a,test,1,2022-03-22 17:07:32.421969,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,...,0.0,0.5,0.500000,45,30,"[1.5919353565641203, -0.39500831467218644, 0.0...",-1.953971,1.591935,0.932385,-0.190665
97,7134685c-4273-42c9-ab2b-feccb85f670a,test,1,2022-03-22 17:07:32.440837,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...",0.0,...,0.0,0.2,0.400000,45,35,"[-1.4973023261712217, -0.6676329849014747, 0.6...",-1.497302,1.297368,0.819600,-0.330674
98,a1fa4dc0-b27d-4f83-89cb-050539d24537,test,1,2022-03-22 17:07:32.459969,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, ...",0.0,...,0.0,0.2,0.400000,45,40,"[-1.423027429671969, 1.6256237382957328, 0.164...",-1.503366,1.625624,0.930722,-0.219893


In [45]:
experiment.save()

In [2]:
experiment = jb.Experiment.load(experiment_id="test", benchmark_id=1)

In [2]:
experiment.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,onehot_violation_min,onehot_violation_mean,onehot_violation_std,one-hot,group_num_people,feas_obj,feas_obj_min,feas_obj_max,feas_obj_std,feas_obj_mean
0,8d5047a1-36d3-4eaf-99ab-18ae5d262c8a,test,1,2022-03-22 17:02:25.218250,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,...,0.0,0.6,0.489898,0,0,[],NaN,NaN,NaN,NaN
1,4df1c746-e8bc-4092-83ef-cf76cff03aa8,test,1,2022-03-22 17:02:25.242991,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",0.0,...,0.0,0.7,0.458258,0,5,[],NaN,NaN,NaN,NaN
2,f6dee37f-38cf-46be-94ef-d7e45f0cc9d4,test,1,2022-03-22 17:02:25.262573,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",0.0,...,0.0,0.5,0.500000,0,10,[],NaN,NaN,NaN,NaN
3,844f4f40-28b5-444d-8a5a-079ee9790629,test,1,2022-03-22 17:02:25.281978,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,...,0.0,0.3,0.458258,0,15,[],NaN,NaN,NaN,NaN
4,2563278e-274a-4f10-834d-7318d4c49ef5,test,1,2022-03-22 17:02:25.302134,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0,...,0.0,0.8,0.400000,0,20,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4c84e289-1b76-4e99-b81c-a609791c5295,test,1,2022-03-22 17:02:27.011786,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.0,...,0.0,0.3,0.458258,45,25,"[-0.14659830772442042, 0.0213302548812574, -0....",-0.381324,1.231911,0.557741,0.274523
96,d351ce7a-5347-43b2-b1e9-ebebb93aa481,test,1,2022-03-22 17:02:27.030588,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...",0.0,...,0.0,0.4,0.489898,45,30,"[2.3763054259337686, 0.28588243257528717, -0.3...",-2.815527,2.376305,1.418742,0.256045
97,fb2c3207-56da-4438-af44-3590deae726a,test,1,2022-03-22 17:02:27.049418,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...",0.0,...,0.0,0.6,0.489898,45,35,"[-0.3224447370059272, -1.7963892257508378, -0....",-1.796389,0.885705,0.746353,-0.125356
98,10582d17-a2da-4960-ad25-e9a7bdd3fff5,test,1,2022-03-22 17:02:27.068245,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...",0.0,...,0.0,0.4,0.489898,45,40,"[-0.721904237296285, -0.06667281086773971, -1....",-1.263789,1.638627,0.825871,0.115037


In [2]:
evaluator = jb.Evaluator(experiment)

In [7]:
def score1(x):
    return x.a + x.b + x.c

evaluator.apply(score1, "score1")
evaluator.table.iloc[:, -5:]


{}


,a,b,c,score2,score1
0,1,2,3,10,6
1,2,3,4,13,9
2,3,4,5,16,12


In [6]:
def score2(x, c1=1, c2=1):
    return x.a + x.b + x.c + c1 + c2
   

# scorer = jb.Evaluator.make_scorer(score2, c1=5, c2=10)
evaluator.apply(score2, "score2", c1=2, c2=2)
evaluator.table.iloc[:, -5:]

{'c1': 2, 'c2': 2}


,onehot_violation_std,a,b,c,score2
0,0.500000,1,2,3,10
1,0.500000,2,3,4,13
2,0.458258,3,4,5,16


In [24]:
response.info["schedule"]["num_sweeps"]

1000

In [25]:
d = {"a": 1, "b": 2}


if "a" in d:
    print("fafafaf")

fafafaf


In [15]:
import jijmodeling as jm
from typing import cast

x = jm.Problem("a")

a = 1
a = cast(str, x)
a.upper()



AttributeError: 'Problem' object has no attribute 'upper'

In [8]:
class A:
    def __init__(self):
        self._run_id = "a"
    
    def reset(self, kind):
        setattr(self, "_bbb_id", kind)
        

obj = A()
obj.reset("bbbbb")
obj._run_id

'a'

In [4]:
import openjij as oj
import inspect
import itertools
import functools
import pandas as pd
from typing import List, Dict, Any, Callable, Union
from jijbench import problems
from jijbench.experiment import Experiment
from jijbench.evaluation import Evaluator
from jijbench.objects.id import ID
from jijbench.objects.dir import ExperimentResultDefaultDir
from jijbench.objects.table import Table
from jijbench.objects.artifact import Artifact


class BenchmarkSolver:
    _default_solver_names = {
                "SASampler": "openjij_sa_sampler_sample",
                "SQASampler": "openjij_sqa_sampler_sample",
                "CSQASampler": "openjij_csqa_sampler_sample"
    }
    
    def __init__(self, solver):
        self.solver = self._parse_solver(solver)
        self._name = self.solver.__name__
        
    def __call__(self, **kwargs):
        parameters = inspect.signature(self.solver).parameters
        kwargs = kwargs if "kwargs" in parameters else {k: v for k, v in kwargs.items() if k in parameters}
        return self.solver(**kwargs)
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        self._name = name
      
    def _parse_solver(self, solver):
        if isinstance(solver, str):
            return getattr(self, self._default_solver_names[solver])
        elif isinstance(solver, Callable):
            return solver
        else:
            return
    
    @staticmethod    
    def on_param_validation(fn):
        @functools.wraps(fn)
        def wrapper(obj, *args, **kwargs):
            solver, = args
            if isinstance(solver, (list, tuple)):
                callable_solvers = []
                for s in solver:
                    callable_solvers.append(BenchmarkSolver(s))
                if isinstance(solver, tuple):
                    callable_solvers = tuple(callable_solvers)
            else:
                callable_solvers = [BenchmarkSolver(solver)]
            fn(obj, callable_solvers)
        return wrapper        

    @staticmethod
    def openjij_sa_sampler_sample(problem, ph_value, feed_dict=None, **kwargs):
        return BenchmarkSolver._sample(oj.SASampler, problem, ph_value, feed_dict, **kwargs)
    
    @staticmethod    
    def openjij_sqa_sampler_sample(problem, ph_value, feed_dict=None, **kwargs):
        return BenchmarkSolver._sample(oj.SQASampler, problem, ph_value, feed_dict, **kwargs)
    
    # エラーが出る
    @staticmethod
    def openjij_csqa_sampler_sample(problem, ph_value, feed_dict=None, **kwargs):
        return BenchmarkSolver._sample(oj.CSQASampler, problem, ph_value, feed_dict, **kwargs)
    
    @staticmethod
    def _sample(sampler, problem, ph_value, feed_dict, **kwargs):
        if feed_dict is None:
            feed_dict = {const_name: 5. for const_name in problem.constraints}
        parameters = inspect.signature(sampler).parameters
        kwargs = {k: w for k, w in kwargs.items() if k in parameters}
        bqm = problem.to_pyqubo(ph_value).compile().to_bqm(feed_dict=feed_dict)
        return sampler(**kwargs).sample(bqm)


class Benchmark:
    def __init__(
                self, 
                params, 
                *,
                benchmark_id: Union[int, str] = None,
                solver: Any = "SASampler", 
                targets=None, 
                id_rules: Union[str, Dict[str, str]]="uuid",
                ):
        self.params = params
        self._id = ID(benchmark_id=benchmark_id)
        self._set_solver(solver)
        if targets is None:
            tsp_instance = problems.tsp.tsp_instance()
            name = tsp_instance.small_list()[0]
            data = tsp_instance.get_instance("small", name)
            self.targets = {problems.tsp.travelling_salesman(): (name, data)}
        else:
            self.targets = targets
        self.id_rules = id_rules
        self._experiments = []
        self._table = Table()
        self._artifact = Artifact()
    
    @property
    def solver(self):
        return self._solver
    
    @BenchmarkSolver.on_param_validation
    def _set_solver(self, solver):
        self._solver = solver
        
    @property
    def experiments(self):
        return self._experiments
    
    @property
    def table(self):
        return self._table.data
    
    @property
    def artifact(self):
        return self._artifact.data
    
    def run(self, solver_ret_names=None):
        keys = list(self.params.keys())
        values = list(self.params.values())
        
        for solver in self.solver:
            for problem, instance in self.targets.items():
                experiment = Experiment(benchmark_id=self._id.benchmark_id)
                instance_name, ph_value = instance  
                kwargs = {"problem": problem, "ph_value": ph_value}
                record = {"problem_name": problem.name, "instance_name": instance_name, "solver": solver.name}
                for r in itertools.product(*values):
                    with experiment:
                        kwargs |= dict([(k, v) for k, v in zip(keys, r)])
                        ret = solver(**kwargs)
                        ret = self._name_solver_ret(ret, solver_ret_names)
                        record |= dict([(k, v) for k, v in zip(keys, r)])
                        record |= ret
                        experiment.store(record)
                        kwargs |= self._parse_ret(ret, kwargs)
                self._table.data = pd.concat([self._table.data, experiment.table])
                self._artifact.data |= experiment.artifact
                self._experiments.append(experiment)

    def compare(self, key, values=None):
        return self._table.data.pivot(columns=key, values=values)
    
    def evaluate(self):
        evaluator = Evaluator(self)
        metrics = evaluator.calc_typical_metrics()
        self._table.data = evaluator.table
        return metrics
    
    @classmethod
    def load(cls,
        *,
        benchmark_id: Union[int, str],
        experiment_id: Union[int, str, List[Union[int, str]]] = None,
        autosave: bool = True,
        save_dir: str = ExperimentResultDefaultDir
        ):
        experiments = []
        table = Table()
        artifact = Artifact()
        experiment_ids = experiment_id if experiment_id else os.listdir(f"{save_dir}/benchmark_{benchmark_id}")
        for experiment_id in experiment_ids:
            experiment = Experiment.load(
                benchmark_id=benchmark_id,
                experiment_id=experiment_id,
                autosave=autosave,
                save_dir=save_dir
                )
            experiments.append(experiment)
            table.data = pd.concat([table.data, experiment.table])
            artifact.data |= experiment.artifact
            
        bench = cls(
            [],
            benchmark_id=benchmark_id
        )
        bench._experiments = experiments
        bench._table = table
        bench._artifacat = artifact
        return bench
    
    
    @staticmethod
    def _name_solver_ret(ret, names):
        if isinstance(ret, dict):
            return ret
        elif isinstance(ret, (list, tuple)):
            names = names if names else [f"solver_return_values[{i}]" for i in range(len(ret))]
            ret = dict(zip(names, ret))
        else:
            ret = {"solver_return_values[0]": ret}
        return ret
    
    @staticmethod
    def _parse_ret(ret,  solver_args):
        return {k: v for k, v in ret.items() if k in solver_args}

         
def func1():
    return 10

def func2(a, b=2, c=3):
    return 1, "a"

""" for sovler in solvers:
    experiment = Experiment():
    for num_reads in [1, 2, 3]:
        for num_sweeps in [10, 20, 30]:
            for a in [2, 3, 4]:
                func(num_reads=num_reads, num_sweeps=num_sweeps, a=a)
                
 """
 
bench = Benchmark(
    {"num_reads": [1, 2, 3], "num_sweeps": [10, 20], "a": [1]},
    solver=[func1, func2],
    benchmark_id="load_test"
)




#solver = [getattr(bench, "openjij_sa_sampler_sample")]
solver = bench.solver

""" feed_dict = {"onehot": 1}
for si in solver:
    print(si(problem=problem, ph_value=ph_value, num_reads=10, num_sweeps=10, a=1000, b=[1, 2, 3]))
    # print(si(bqm=2, a=1, b=2))
    #print(s._default_params)
    print() """

 
bench.run()
bench.evaluate()
# bench.table.dropna(how="all", axis=1)
bench.table


,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,obj_mean,obj_std,num_occurances,num_reads,num_sweeps,num_feasible,num_samples,sampling_time,execution_time,problem_name,instance_name,solver,a,solver_return_values[0],solver_return_values[1],success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,e3b6342f-d5c6-4510-be1d-40b46ce4cfad,112d9048-295e-4479-8152-82d02a52a1a8,load_test,2022-03-24 06:05:40.612961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4c995bdb-50e6-4910-a43f-e1411f724c82,112d9048-295e-4479-8152-82d02a52a1a8,load_test,2022-03-24 06:05:40.624789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7799c238-a2cf-40b0-a33c-313a6a5e1c01,112d9048-295e-4479-8152-82d02a52a1a8,load_test,2022-03-24 06:05:40.632190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,10,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e870ecc8-e550-4ec1-92dd-d75a68c1d73d,112d9048-295e-4479-8152-82d02a52a1a8,load_test,2022-03-24 06:05:40.639488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22279e64-8c29-4963-baa8-54571ac57f34,112d9048-295e-4479-8152-82d02a52a1a8,load_test,2022-03-24 06:05:40.647092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,10,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cc7f11ea-a0e0-42b9-8ebb-952c77b9a1ba,112d9048-295e-4479-8152-82d02a52a1a8,load_test,2022-03-24 06:05:40.654404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,20,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ef8a3f02-bcda-423d-9ad3-966d66a1d2c8,1aa6c3de-3c5a-47ea-8732-2be1cce74198,load_test,2022-03-24 06:05:40.663055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a,NaN,NaN,NaN,NaN,NaN,NaN
1,44b189f1-d778-4db8-8330-c1b37313cd30,1aa6c3de-3c5a-47ea-8732-2be1cce74198,load_test,2022-03-24 06:05:40.674346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a,NaN,NaN,NaN,NaN,NaN,NaN
2,7e713599-d543-48dd-947c-1010f8d06c3a,1aa6c3de-3c5a-47ea-8732-2be1cce74198,load_test,2022-03-24 06:05:40.681831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,10,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a,NaN,NaN,NaN,NaN,NaN,NaN
3,70401889-55f0-4814-a611-f3cfed51a930,1aa6c3de-3c5a-47ea-8732-2be1cce74198,load_test,2022-03-24 06:05:40.689230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
bench.compare("solver", ["num_reads", "num_sweeps", "TTS(feasible)"])

num_reads       num_sweeps       TTS(feasible)      
solver     func1 func2      func1 func2         func1 func2
0              1     1         10    10           NaN   NaN
1              1     1         20    20           NaN   NaN
2              2     2         10    10           NaN   NaN
3              2     2         20    20           NaN   NaN
4              3     3         10    10           NaN   NaN
5              3     3         20    20           NaN   NaN

In [15]:
# for文のイメージ
bench = Benchmark(
    {"num_reads": [1, 2, 3], "num_sweeps": [10, 20]},
)

for num_reads in [1, 2, 3]:
    for num_sweeps in [10, 20]:
        print(num_reads, num_sweeps)

1 10
1 20
2 10
2 20
3 10
3 20


In [13]:
from jijbench.evaluation import Evaluator

evaluator = Evaluator(bench)
evaluator.calc_typical_metrics()


,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
bench.compare("solver")

run_id  \
solver                                 func1   
0       d9ce7826-9310-4813-b598-eea3e99d3ab4   
1       7f147324-4c86-415c-83de-e9e77ed074e9   

                                              \
solver                                 func2   
0       8f8831bd-e7e9-4215-90ab-0b029ee9ca9f   
1       f64a9c57-12c2-4058-bb84-1d24c0223a10   

                               experiment_id  \
solver                                 func1   
0       457df92a-11b9-4650-8c45-a2c8d12d3e46   
1       457df92a-11b9-4650-8c45-a2c8d12d3e46   

                                             benchmark_id             \
solver                                 func2        func1      func2   
0       a7be4639-a5e3-4a82-9ab6-9f1d71589a18    load_test  load_test   
1       a7be4639-a5e3-4a82-9ab6-9f1d71589a18    load_test  load_test   

                         timestamp                             energy        \
solver                       func1                       func2  func1 func2   
0       2022-03-24 03:55:50.864838  2022-03-24 03:55:50.885135    NaN   NaN   
1       2022-03-24 03:55:50.876456  2022-03-24 03:55:50.896360    NaN   NaN   

        ...         problem_name                      instance_name         \
solver  ...                func1                func2         func1  func2   
0       ...  travelling_salesman  travelling_salesman         ftv35  ftv35   
1       ...  travelling_salesman  travelling_salesman         ftv35  ftv35   

           a       solver_return_values[0]       solver_return_values[1]        
solver func1 func2                   func1 func2                   func1 func2  
0        1.0   1.0                    10.0   1.0                     NaN     a  
1        1.0   1.0                    10.0   1.0                     NaN     a  

[2 rows x 48 columns]

In [31]:
bench.artifact

{'b91d393e-7628-4a65-b452-9b206fd7d640': {'problem_name': 'travelling_salesman',
  'instance_name': 'ftv35',
  'solver': 'func1',
  'num_reads': 2,
  'num_sweeps': 10,
  'a': 1,
  'solver_return_values[0]': 10},
 '75930973-198f-4eb4-ace6-4b275f7d8ece': {'problem_name': 'travelling_salesman',
  'instance_name': 'ftv35',
  'solver': 'func1',
  'num_reads': 2,
  'num_sweeps': 10,
  'a': 1,
  'solver_return_values[0]': 10},
 '1c7b5bca-9039-4a4a-845a-375f957f43a2': {'problem_name': 'travelling_salesman',
  'instance_name': 'ftv35',
  'solver': 'func2',
  'num_reads': 2,
  'num_sweeps': 10,
  'a': 1,
  'solver_return_values[0]': 1,
  'solver_return_values[1]': 'a'},
 'bb6ed114-aa16-4e0b-8b8e-b9bfb4811384': {'problem_name': 'travelling_salesman',
  'instance_name': 'ftv35',
  'solver': 'func2',
  'num_reads': 2,
  'num_sweeps': 10,
  'a': 1,
  'solver_return_values[0]': 1,
  'solver_return_values[1]': 'a'}}

In [3]:
bench = Benchmark.load(
    benchmark_id="load_test",
    # experiment_id=["ad4d6caa-e2b7-4120-9c7a-faa1e99759ec"]
)

In [4]:
bench.table

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,num_feasible,num_samples,sampling_time,execution_time,problem_name,instance_name,solver,a,solver_return_values[0],solver_return_values[1]
0,04b88efb-70de-424b-a88d-91922024500b,b3126a74-59c6-4765-b575-3ccfd7cccd88,load_test,2022-03-24 03:49:32.962150,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
1,66227fec-854d-4e56-9a62-f33ca3487489,b3126a74-59c6-4765-b575-3ccfd7cccd88,load_test,2022-03-24 03:49:32.973342,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,8a9c68c1-3e0b-4e6a-bf22-638ed32077e9,bba95956-2f62-4469-8331-6386c796eb09,load_test,2022-03-24 03:49:32.941911,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN
1,c4640f81-3552-439b-81c9-f027a9971714,bba95956-2f62-4469-8331-6386c796eb09,load_test,2022-03-24 03:49:32.953179,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func1,1.0,10.0,NaN


In [61]:
df = pd.DataFrame()
df["a"] = [1] * 10
df.to_csv("df.csv")

In [42]:
# dfs = dict([g for g in bench.table.groupby(["solver"])])
# df = pd.concat([dfs["func1"].reset_index(drop=True), dfs["func2"].reset_index(drop=True)], axis=1)

#for gname, df in bench.table.groupby(["solver"]):
#    columns = 


func1
func2


In [8]:
bench.table.pivot(columns="solver")

TypeError: pivot() missing 1 required argument: 'columns'

In [40]:
bench.table.groupby(["solver"]).count()

/tmp/ipykernel_21067/1754180164.py:1: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  bench.table.groupby(["solver"], axis=1).count()


ValueError: cannot reindex on an axis with duplicate labels

In [39]:
pd.concat([dfs["func1"], dfs["func2"]], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [25]:
dfs["func2"]

,run_id,experiment_id,benchmark_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,num_feasible,num_samples,sampling_time,execution_time,problem_name,instance_name,solver,a,solver_return_values[0],solver_return_values[1]
0,ecfbd4cd-be16-4c44-8967-e95dd8753308,f0d9ed6a-7d49-4c96-aee0-e0eb3627d2ae,load_test,2022-03-24 01:49:29.052768,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,c8515773-fbca-4dc3-a5c0-2f73aa6080e8,f0d9ed6a-7d49-4c96-aee0-e0eb3627d2ae,load_test,2022-03-24 01:49:29.065020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,51384c28-d6a8-4cee-8ddb-ef292dba0162,2fc117a9-081c-49b7-9505-0dac71f61665,load_test,2022-03-24 02:25:59.375080,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,9f43d98c-cd63-460a-9214-26e725e5c712,2fc117a9-081c-49b7-9505-0dac71f61665,load_test,2022-03-24 02:25:59.386233,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,e50d2245-4ecc-49af-a376-d30fc96d6d4b,b3e323c2-22ca-4919-bb4e-84e379502020,load_test,2022-03-24 02:17:30.822915,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,3ec003d4-2eef-40f0-bd86-2d31ca4b08d0,b3e323c2-22ca-4919-bb4e-84e379502020,load_test,2022-03-24 02:17:30.834130,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,1c7b5bca-9039-4a4a-845a-375f957f43a2,c9e1f2b5-8179-4ec8-bd47-be3e35c2f533,load_test,2022-03-24 02:21:47.165723,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,bb6ed114-aa16-4e0b-8b8e-b9bfb4811384,c9e1f2b5-8179-4ec8-bd47-be3e35c2f533,load_test,2022-03-24 02:21:47.177103,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,b733bfbc-9056-4577-ab39-96ef0f429f72,57c9e1c5-2684-4ca5-bffd-f377b4134383,load_test,2022-03-24 01:49:10.295616,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a
0,378aae9a-80f3-47a6-b975-b1b624a09343,57c9e1c5-2684-4ca5-bffd-f377b4134383,load_test,2022-03-24 01:49:10.306735,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,travelling_salesman,ftv35,func2,1.0,1.0,a


In [26]:
x = {"a": 1}
y = {"b": 1}
c = {}



ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [50]:
func2.__name__

'func2'

In [49]:
a = {"a": 1, "b": 2, "c": 3}
x = {"a": 3}

a |= x
a

{'a': 3, 'b': 2, 'c': 3}

In [32]:
names = ["a"]
"".join(names)

'a'

In [6]:
x = {"a": 1, "b": 2, "c": 3}
print(x.pop("a"))
print(x)

1
{'b': 2, 'c': 3}


In [2]:
def func(x, a=1):
    pass

func(b=1)

TypeError: func() got an unexpected keyword argument 'b'

In [92]:
a = 1
# a.__name__
print(dir(a))


def func():
    return

class A:
    a = 1

func().__str__()
str(A())


['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'as_integer_ratio', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']


'<__main__.A object at 0x7ff705ecd040>'

In [82]:
names = ["a", "b"]
ret = [1, 2]

print(list(zip(names, ret)))
print(dict(zip(names, ret)))

[('a', 1), ('b', 2)]
{'a': 1, 'b': 2}


In [79]:
isinstance((1,), (list, tuple))

True

In [36]:
def func1(*args):
    return 1, "a"

def func2(*args):
    return 1

values = list(bench.params.values())

x = func(*values)
x


SyntaxError: starred assignment target must be in a list or tuple (22539421.py, line 6)

In [14]:
import pandas as pd

df = pd.DataFrame(columns=["a", "b", "c"])
df.loc[0] = np.nan
df

,a,b,c
0,NaN,NaN,NaN


In [425]:
instance = problems.tsp.get_instance().small_list()

instance

['ftv35', 'ftv38', 'br17', 'ftv33', 'p43']

In [404]:
import inspect

def func(x=1, y=2, **kwargs):
    pass

args = inspect.signature(func).parameters
"x" in args

True

In [368]:
problem

In [369]:
ph_value

{'d': 2}

In [361]:
dir(Benchmark)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_set_solver',
 'experiments',
 'openjij_sa_sampler_sample',
 'run',
 'solver']

In [354]:
import jijbench as jb

problem = jb.problems.tsp.travelling_salesman()
problem

In [341]:
sampler = getattr(oj.sampler, "SASampler")()
sampler.sample

<bound method SASampler.sample of <openjij.sampler.sa_sampler.SASampler object at 0x7f15393e1a90>>

In [314]:
isinstance(oj.SASampler.sample, Callable)

True

In [297]:
import itertools

x = {"a": [1, 2, 3], "b": [4, 5]}
print(list(x.keys()))
print(list(x.values()))

a = [1, 2, 3]
b = [4, 5, 6]
c = [7, 8, 9]

print(*list(x.values()))
l = itertools.product(*list(x.values()))
for li in l:
    print(*li)

['a', 'b']
[[1, 2, 3], [4, 5]]
[1, 2, 3] [4, 5]
1 4
1 5
2 4
2 5
3 4
3 5


In [220]:
None()

<>:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_14354/744963307.py:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
  None()
/tmp/ipykernel_14354/744963307.py:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
  None()
/tmp/ipykernel_14354/744963307.py:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
  None()


TypeError: 'NoneType' object is not callable

In [81]:
bench = Benchmark(
    {"num_reads": [1, 2, 3]},
)

print(bench.solver)

_wrapper
<function _param_validation.<locals>._wrapper.<locals>._solver at 0x7f153a239e50>


In [10]:
getattr(oj.sampler, "SASampler")

openjij.sampler.sa_sampler.SASampler

In [1]:
import openjij as oj
import jijzept as jz
import jijmodeling as jm

import sys
sys.path.append("../")

import jijbench as jb

def generate_problem():
    d = jm.Placeholder("d", dim=1)
    x = jm.Binary("x", shape=(d.shape[0].set_latex("n")))
    i = jm.Element("i", d.shape[0])
    problem = jm.Problem("simple_problem")
    problem += jm.Sum(i, d[i] * x[i])
    problem += jm.Constraint("onehot", jm.Sum(i, d[i] * x[i]) == 1)
    return problem

def sa_sampler(problem, instance_data, multipliers):
    sampler = oj.SASampler()
    bqm = problem.to_pyqubo(instance_data).compile().to_bqm(feed_dict=multipliers)
    res = sampler.sample(bqm, num_reads=10)
    decoded_samples = problem.decode(res, instance_data)
    return res, decoded_samples

problem = generate_problem()
instance_data = {"d": [1, 2, 3]}

bench = jb.Benchmark(
    params={"multipliers": [{"onehot": 1}, {"onehot": 2}, {"onehot": 3}]},
    solver=sa_sampler,
    problem=problem,
    instance_data=instance_data
)
bench.run()

/tmp/ipykernel_30156/2778055201.py:21: DeprecationWarning: Call to deprecated method to_pyqubo. (This method is deprecated) -- Deprecated since version v0.9.37.
  bqm = problem.to_pyqubo(instance_data).compile().to_bqm(feed_dict=multipliers)
/tmp/ipykernel_30156/2778055201.py:21: DeprecationWarning: Call to deprecated method to_pyqubo. (This method is deprecated) -- Deprecated since version v0.9.37.
  bqm = problem.to_pyqubo(instance_data).compile().to_bqm(feed_dict=multipliers)
/tmp/ipykernel_30156/2778055201.py:21: DeprecationWarning: Call to deprecated method to_pyqubo. (This method is deprecated) -- Deprecated since version v0.9.37.
  bqm = problem.to_pyqubo(instance_data).compile().to_bqm(feed_dict=multipliers)


In [2]:
bench.table

,benchmark_id,experiment_id,run_id,timestamp,energy,energy_min,energy_mean,energy_std,objective,obj_min,...,execution_time,problem_name,instance_data_name,opt_value,solver,multipliers,onehot_violations,onehot_violation_min,onehot_violation_mean,onehot_violation_std
0,174fa262-aa10-456d-b169-bb51cc6632b6,a9d50237-652c-48f9-a867-2d96135a64bb,2678b813-7474-4326-b814-8c591882d026,2022-07-28 11:46:19.042226,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...",0.0,...,273.746345,simple_problem,Unnamed[0],NaN,sa_sampler,{'onehot': 1},"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,0.3,0.458258
1,174fa262-aa10-456d-b169-bb51cc6632b6,a9d50237-652c-48f9-a867-2d96135a64bb,94255d78-d2b0-4a14-b17e-a408f94f0ef5,2022-07-28 11:46:19.079266,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,...,292.736758,simple_problem,Unnamed[0],NaN,sa_sampler,{'onehot': 2},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000
2,174fa262-aa10-456d-b169-bb51cc6632b6,a9d50237-652c-48f9-a867-2d96135a64bb,e7e44676-3ccf-4d9d-9e08-19b3b7889d01,2022-07-28 11:46:19.107378,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,1.0,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.0,...,290.536508,simple_problem,Unnamed[0],NaN,sa_sampler,{'onehot': 3},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,0.000000


In [3]:
bench.table["obj_min"]

0    0.0
1    1.0
2    1.0
Name: obj_min, dtype: object

In [2]:
bench.evaluate(0)

nan


,success_probability,feasible_rate,residual_energy,TTS(optimal),TTS(feasible),TTS(derived)
0,NaN,0.4,NaN,NaN,2434.950613,33.858016
1,NaN,1.0,NaN,NaN,36.394033,36.394033
2,NaN,1.0,NaN,NaN,35.863758,35.863758
